<a href="https://colab.research.google.com/github/ryu622/gnn-counterattack-xai-v2/blob/feat%2Fnew-file/GAT_CounterAttack_Prediction_Train_Scientific8_V6_take3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

最終的に卒論で使用した提案モデル。
ヒューリスティックな制約を加えた提案手法１（GATを拡張して作成。アテンション係数の可視化、順列特徴重要度、GNNExplainerまで実装）

In [ ]:
#シード値
import random
import os
import numpy as np
import torch

def set_seed(seed=42):
    # Python自体の乱数固定
    random.seed(seed)
    # OS環境の乱数固定
    os.environ['PYTHONHASHSEED'] = str(seed)
    # Numpyの乱数固定
    np.random.seed(seed)
    # PyTorchの乱数固定
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) # マルチGPUの場合
    # 計算の決定論的挙動を強制（これを入れると少し遅くなることがありますが、再現性は完璧になります）
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# 好きな数字（42が一般的）で固定
set_seed(42)

In [ ]:
#GoogleDriveをマウント

from google.colab import drive

# Google Driveを仮想ファイルシステムにマウント
drive.mount('/content/drive')

In [ ]:
# 必須モジュールのインポート
!pip install torch_geometric

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch import optim

import pandas as pd
import os
from glob import glob
import numpy as np
import re
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

# 表示設定
np.set_printoptions(suppress=True, precision=3)
pd.set_option('display.precision', 3)    # 小数点以下の表示桁
pd.set_option('display.max_rows', 50)   # 表示する行数の上限
pd.set_option('display.max_columns', 15)  # 表示する列数の上限
%precision 3

使用データは以前までと同様

In [ ]:
import torch
import numpy as np
from torch_geometric.loader import DataLoader
from sklearn.metrics import classification_report
from collections import defaultdict
import random

# ==========================================
# 1. 補助関数: 試合単位のアンダーサンプリング
# ==========================================
def balance_dataset_by_undersampling(data_list):
    """
    シークエンス単位で1:1に調整。訓練データにのみ適用。
    """
    seq_groups = defaultdict(list)
    for d in data_list:
        sid = int(d.sequence_id.item()) if torch.is_tensor(d.sequence_id) else int(d.sequence_id)
        seq_groups[sid].append(d)

    l0_groups, l1_groups = [], []
    for sid, frames in seq_groups.items():
        label = int(frames[0].y.item())
        if label == 0: l0_groups.append(frames)
        else: l1_groups.append(frames)

    # 少ない方（成功）に合わせて失敗を削る
    min_size = min(len(l0_groups), len(l1_groups))
    sampled_l0 = random.sample(l0_groups, min_size)
    sampled_l1 = l1_groups # 成功は全数保持

    balanced_list = [frame for group in (sampled_l0 + sampled_l1) for frame in group]
    random.shuffle(balanced_list)

    print(f"    [Sampling] Success Seqs: {len(sampled_l1)} | Failure Seqs: {len(sampled_l0)} | Total Frames: {len(balanced_list)}")
    return balanced_list



モデルの定義（PIGNNのオリジナルクラス）

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing, global_mean_pool
from torch_geometric.utils import softmax
import os

# ==========================================
# 1. 前処理関数の定義
# ==========================================
def preprocess_batch(data, device):
    data.x = data.x.float()
    data.pos = data.pos.float()
    data.vel = data.vel.float()
    return data.to(device)

# ==========================================
# 2. モデル定義（可視化対応 & tau）
# ==========================================
class PIGNNLayer(MessagePassing):
    def __init__(self, in_channels, out_channels, tau=1.5):
        super(PIGNNLayer, self).__init__(aggr='add')
        self.tau = tau
        self.lin = nn.Linear(in_channels, out_channels)
        self.att = nn.Parameter(torch.Tensor(1, out_channels * 2))
        nn.init.xavier_uniform_(self.att)
        self._last_att = None  # 可視化用に保存する変数

    def forward(self, x, edge_index, pos, vel, return_attention=False):
        h = self.lin(x)
        # チームID(index 6)をメッセージパッシングに渡す
        out = self.propagate(edge_index, x=h, pos=pos, vel=vel, team=x[:, 6:7])

        if return_attention:
            # message内で保存されたアテンション係数を返す
            return out, (edge_index, self._last_att)
        return out

    def message(self, x_i, x_j, pos_i, pos_j, vel_i, vel_j, edge_index_i, team_i, team_j):
        # 未来位置予測（tau=0.2）
        pos_i_pred = pos_i + vel_i * self.tau
        pos_j_pred = pos_j + vel_j * self.tau
        dist_future = torch.norm(pos_i_pred - pos_j_pred, dim=-1, keepdim=True)
        physics_bias = torch.exp(-dist_future / 2.0)

        # チーム関係バイアス
        is_teammate = (team_i == team_j).float()
        team_bias = torch.where(is_teammate > 0.5, 0.5, -0.5)

        # アテンション重み計算
        alpha = torch.cat([x_i, x_j], dim=-1)
        alpha = (alpha * self.att).sum(dim=-1, keepdim=True)
        alpha = softmax(F.leaky_relu(alpha) + physics_bias + team_bias, edge_index_i)

        # 可視化用にアテンション係数を記録
        self._last_att = alpha
        return alpha * x_j

class PIGNNClassifier(nn.Module):
    def __init__(self, hidden_channels=64):
        super(PIGNNClassifier, self).__init__()
        self.conv1 = PIGNNLayer(7, hidden_channels, tau=0.2)
        self.conv2 = PIGNNLayer(hidden_channels, hidden_channels, tau=0.2)
        self.lin = nn.Linear(hidden_channels, 2)

    def forward(self, data, return_attention=False):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        pos, vel = data.pos, data.vel

        if return_attention:
            # 1層目（物理的特徴が最も強く出る層）のアテンションを取得
            x, (edge_idx_out, att_weights) = self.conv1(x, edge_index, pos, vel, return_attention=True)
            x = F.elu(x)
            x = self.conv2(x, edge_index, pos, vel)
        else:
            x = F.elu(self.conv1(x, edge_index, pos, vel))
            x = self.conv2(x, edge_index, pos, vel)

        x_pool = global_mean_pool(x, batch)
        logits = F.log_softmax(self.lin(x_pool), dim=1)

        if return_attention:
            return logits, (edge_idx_out, att_weights)
        return logits

# ==========================================
# 3. 理論修正：集団運動学的制約（L_phys）
# ==========================================
def pignn_theoretical_loss(output, target, data, alpha=0.1):
    weights = torch.tensor([1.0, 3.3], device=output.device)
    loss_task = F.nll_loss(output, target, weight=weights)

    probs = torch.exp(output)[:, 1]

    # --- 攻撃選手のみの推進力を計算（index 6 を使用） ---
    atk_mask = (data.x[:, 6] == 1.0)
    all_vxs = data.vel[:, 0]

    batch_size = output.size(0)
    avg_vxs_atk = []

    for i in range(batch_size):
        mask = (data.batch == i) & atk_mask
        if mask.any():
            avg_vxs_atk.append(torch.mean(all_vxs[mask]))
        else:
            avg_vxs_atk.append(torch.tensor(0.0, device=output.device))

    avg_vxs_atk = torch.stack(avg_vxs_atk)

    # 物理損失：Success確率 × ReLU(-攻撃側の平均vx)
    loss_phys = torch.mean(probs * torch.relu(-avg_vxs_atk))

    total_loss = loss_task + (alpha * loss_phys)
    return total_loss, loss_task, loss_phys

# ==========================================
# 4. 学習・評価ループ
# ==========================================
def train_pignn_epoch_fixed(model, loader, optimizer, device, alpha_p):
    model.train()
    total_loss, total_phys = 0, 0
    current_alpha = alpha_p

    for data in loader:
        data = preprocess_batch(data, device)
        optimizer.zero_grad()
        out = model(data)
        loss, _, l_phys = pignn_theoretical_loss(out, data.y.view(-1), data, alpha=current_alpha)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
        total_phys += l_phys.item() * data.num_graphs

    return total_loss / len(loader.dataset), total_phys / len(loader.dataset), current_alpha

def test_pignn(model, loader, device):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data in loader:
            data = preprocess_batch(data, device)
            out = model(data)
            pred = out.argmax(dim=1)
            correct += (pred == data.y.view(-1)).sum().item()
    return correct / len(loader.dataset)

In [ ]:
# ==========================================
# 2. メイン実行セクション (CVループ) - 学習曲線記録版
# ==========================================
import matplotlib.pyplot as plt # 追加

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 論文同様、固定値で評価（0, 1.0, 10.0など）
FIXED_ALPHA = 50.0

v16_load_path = "/content/drive/MyDrive/GNN_Football_Analysis/Processed_Data/gnn_data_v17_final.pt"
print(f"CV用マスターデータをロード中: {v16_load_path}")
checkpoint = torch.load(v16_load_path, weights_only=False)
all_data_list = checkpoint['all_data']

# --- 【修正】推測ロジックを削除し、刻印されたIDを直接取得 ---
match_ids = sorted(list(set([int(d.match_id.item()) for d in all_data_list])))
print(f"検出された試合ID: {match_ids} (計 {len(match_ids)} 試合)")

# ハイパーパラメータ
EPOCHS_CV =100
LR = 0.0005
cv_final_reports = []
best_overall_f1 = 0

# --- 【追加】全CVの学習ログを保存する箱 ---
all_cv_history = []

print(f"PIGNN クロスバリデーション開始 (Alpha={FIXED_ALPHA} 固定モード)\n")

for test_match in match_ids:
    print(f"\n{'='*60}")
    print(f" 🌀 Round: Match {test_match} をテストに使用")
    print(f"{'='*60}")

    # 1. 刻印された match_id を信じて切り分け
    test_indices = [d for d in all_data_list if int(d.match_id.item()) == test_match]
    train_candidates = [d for d in all_data_list if int(d.match_id.item()) != test_match]

    # 2. 訓練データのみ1:1アンダーサンプリング
    cv_train_set = balance_dataset_by_undersampling(train_candidates)

    cv_train_loader = DataLoader(cv_train_set, batch_size=32, shuffle=True)
    cv_test_loader = DataLoader(test_indices, batch_size=32, shuffle=False)

    cv_model = PIGNNClassifier(hidden_channels=64).to(device)
    cv_optimizer = torch.optim.Adam(cv_model.parameters(), lr=LR)

    # --- 【追加】このRoundのログを保持する辞書 ---
    round_history = {'total_loss': [], 'physics_loss': []}

    # 3. 学習ループ (固定Alpha版)
    for epoch in range(1, EPOCHS_CV + 1):
        loss, phys, _ = train_pignn_epoch_fixed(cv_model, cv_train_loader, cv_optimizer, device, FIXED_ALPHA)

        # --- 【追加】ログを記録 ---
        round_history['total_loss'].append(loss)
        round_history['physics_loss'].append(phys)

        if epoch % 10 == 0 or epoch == 1:
            print(f"  Epoch {epoch:02d} | Loss: {loss:.4f} | Phys_L: {phys:.6f}")

    all_cv_history.append(round_history) # 追加

    # 4. 評価
    cv_model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for data in cv_test_loader:
            data = data.to(device)
            out = cv_model(data)
            y_true.extend(data.y.view(-1).cpu().numpy())
            y_pred.extend(out.argmax(dim=1).cpu().numpy())

    # スコア集計
    report = classification_report(y_true, y_pred, output_dict=True, zero_division=0)
    current_f1 = report['1']['f1-score']

    cv_final_reports.append({
        'match': test_match,
        'recall': report['1']['recall'],
        'precision': report['1']['precision'],
        'f1': current_f1
    })

    # --- α別にフォルダを分けて整理保存 ---
    import os
    base_model_dir = "/content/drive/MyDrive/GNN_Football_Analysis/Models"
    alpha_dir = os.path.join(base_model_dir, f"alpha_{str(FIXED_ALPHA).replace('.', '_')}_take3")
    os.makedirs(alpha_dir, exist_ok=True)

    model_filename = f'pignn_testmatch_{test_match}.pth'
    model_path = os.path.join(alpha_dir, model_filename)
    torch.save(cv_model.state_dict(), model_path)
    print(f" >> [Alpha {FIXED_ALPHA}] Match {test_match} weight saved.")

    if current_f1 > best_overall_f1:
        best_overall_f1 = current_f1
        best_model_path = os.path.join(alpha_dir, f'best_overall_alpha_{FIXED_ALPHA}.pth')
        torch.save(cv_model.state_dict(), best_model_path)
        print(f"  [Alpha {FIXED_ALPHA}] New Best Model Saved: F1={best_overall_f1:.4f}")

    print(f" >> Result: Recall={report['1']['recall']:.4f}, Precision={report['1']['precision']:.4f}, F1={current_f1:.4f}")

# 5. 最終集計
print(f"\n\n{'#'*60}\n  Alpha={FIXED_ALPHA} CV 最終平均結果\n{'#'*60}")
avg_recall = np.mean([r['recall'] for r in cv_final_reports])
avg_f1 = np.mean([r['f1'] for r in cv_final_reports])
avg_precision = np.mean([r['precision'] for r in cv_final_reports])

print(f"\n[OVERALL] Avg Success Recall:     {avg_recall:.4f}")
print(f"[OVERALL] Avg Success Precision: {avg_precision:.4f}")
print(f"[OVERALL] Avg Success F1-score:  {avg_f1:.4f}")

# ==========================================
# 6. 学習曲線の描画 (CV平均) - 【追加セクション】
# ==========================================
avg_total_loss = np.mean([h['total_loss'] for h in all_cv_history], axis=0)
avg_phys_loss = np.mean([h['physics_loss'] for h in all_cv_history], axis=0)
epochs_range = range(1, EPOCHS_CV + 1)

fig, ax1 = plt.subplots(figsize=(10, 6))

ax1.set_xlabel('Epochs')
ax1.set_ylabel('Total Training Loss', color='tab:blue')
ax1.plot(epochs_range, avg_total_loss, color='tab:blue', lw=2, label='Total Loss (Avg)')
ax1.tick_params(axis='y', labelcolor='tab:blue')
ax1.grid(True, linestyle='--', alpha=0.5)

ax2 = ax1.twinx()
ax2.set_ylabel('Physics Loss ($L_{phys}$)', color='tab:red')
ax2.plot(epochs_range, avg_phys_loss, color='tab:red', lw=2, linestyle=':', label='Physics Loss (Avg)')
ax2.tick_params(axis='y', labelcolor='tab:red')

plt.title(f'PIGNN Learning Curve (CV Average, Alpha={FIXED_ALPHA})')
fig.tight_layout()
plt.show()

In [ ]:
# ==========================================
# 2. メイン実行セクション (CVループ) - 学習曲線記録版
# ==========================================
import matplotlib.pyplot as plt # 追加

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 論文同様、固定値で評価（0, 1.0, 10.0など）
FIXED_ALPHA = 0

v16_load_path = "/content/drive/MyDrive/GNN_Football_Analysis/Processed_Data/gnn_data_v17_final.pt"
print(f"CV用マスターデータをロード中: {v16_load_path}")
checkpoint = torch.load(v16_load_path, weights_only=False)
all_data_list = checkpoint['all_data']

# --- 【修正】推測ロジックを削除し、刻印されたIDを直接取得 ---
match_ids = sorted(list(set([int(d.match_id.item()) for d in all_data_list])))
print(f"検出された試合ID: {match_ids} (計 {len(match_ids)} 試合)")

# ハイパーパラメータ
EPOCHS_CV =100
LR = 0.0005
cv_final_reports = []
best_overall_f1 = 0

# --- 【追加】全CVの学習ログを保存する箱 ---
all_cv_history = []

print(f"PIGNN クロスバリデーション開始 (Alpha={FIXED_ALPHA} 固定モード)\n")

for test_match in match_ids:
    print(f"\n{'='*60}")
    print(f"  Round: Match {test_match} をテストに使用")
    print(f"{'='*60}")

    # 1. 刻印された match_id を信じて切り分け
    test_indices = [d for d in all_data_list if int(d.match_id.item()) == test_match]
    train_candidates = [d for d in all_data_list if int(d.match_id.item()) != test_match]

    # 2. 訓練データのみ1:1アンダーサンプリング
    cv_train_set = balance_dataset_by_undersampling(train_candidates)

    cv_train_loader = DataLoader(cv_train_set, batch_size=32, shuffle=True)
    cv_test_loader = DataLoader(test_indices, batch_size=32, shuffle=False)

    cv_model = PIGNNClassifier(hidden_channels=64).to(device)
    cv_optimizer = torch.optim.Adam(cv_model.parameters(), lr=LR)

    # --- 【追加】このRoundのログを保持する辞書 ---
    round_history = {'total_loss': [], 'physics_loss': []}

    # 3. 学習ループ (固定Alpha版)
    for epoch in range(1, EPOCHS_CV + 1):
        loss, phys, _ = train_pignn_epoch_fixed(cv_model, cv_train_loader, cv_optimizer, device, FIXED_ALPHA)

        # --- 【追加】ログを記録 ---
        round_history['total_loss'].append(loss)
        round_history['physics_loss'].append(phys)

        if epoch % 10 == 0 or epoch == 1:
            print(f"  Epoch {epoch:02d} | Loss: {loss:.4f} | Phys_L: {phys:.6f}")

    all_cv_history.append(round_history) # 追加

    # 4. 評価
    cv_model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for data in cv_test_loader:
            data = data.to(device)
            out = cv_model(data)
            y_true.extend(data.y.view(-1).cpu().numpy())
            y_pred.extend(out.argmax(dim=1).cpu().numpy())

    # スコア集計
    report = classification_report(y_true, y_pred, output_dict=True, zero_division=0)
    current_f1 = report['1']['f1-score']

    cv_final_reports.append({
        'match': test_match,
        'recall': report['1']['recall'],
        'precision': report['1']['precision'],
        'f1': current_f1
    })

    # --- α別にフォルダを分けて整理保存 ---
    import os
    base_model_dir = "/content/drive/MyDrive/GNN_Football_Analysis/Models"
    alpha_dir = os.path.join(base_model_dir, f"alpha_{str(FIXED_ALPHA).replace('.', '_')}_take3")
    os.makedirs(alpha_dir, exist_ok=True)

    model_filename = f'pignn_testmatch_{test_match}.pth'
    model_path = os.path.join(alpha_dir, model_filename)
    torch.save(cv_model.state_dict(), model_path)
    print(f" >> [Alpha {FIXED_ALPHA}] Match {test_match} weight saved.")

    if current_f1 > best_overall_f1:
        best_overall_f1 = current_f1
        best_model_path = os.path.join(alpha_dir, f'best_overall_alpha_{FIXED_ALPHA}.pth')
        torch.save(cv_model.state_dict(), best_model_path)
        print(f"  [Alpha {FIXED_ALPHA}] New Best Model Saved: F1={best_overall_f1:.4f}")

    print(f" >> Result: Recall={report['1']['recall']:.4f}, Precision={report['1']['precision']:.4f}, F1={current_f1:.4f}")

# 5. 最終集計
print(f"\n\n{'#'*60}\n  Alpha={FIXED_ALPHA} CV 最終平均結果\n{'#'*60}")
avg_recall = np.mean([r['recall'] for r in cv_final_reports])
avg_f1 = np.mean([r['f1'] for r in cv_final_reports])
avg_precision = np.mean([r['precision'] for r in cv_final_reports])

print(f"\n[OVERALL] Avg Success Recall:     {avg_recall:.4f}")
print(f"[OVERALL] Avg Success Precision: {avg_precision:.4f}")
print(f"[OVERALL] Avg Success F1-score:  {avg_f1:.4f}")

# ==========================================
# 6. 学習曲線の描画 (CV平均) - 【追加セクション】
# ==========================================
avg_total_loss = np.mean([h['total_loss'] for h in all_cv_history], axis=0)
avg_phys_loss = np.mean([h['physics_loss'] for h in all_cv_history], axis=0)
epochs_range = range(1, EPOCHS_CV + 1)

fig, ax1 = plt.subplots(figsize=(10, 6))

ax1.set_xlabel('Epochs')
ax1.set_ylabel('Total Training Loss', color='tab:blue')
ax1.plot(epochs_range, avg_total_loss, color='tab:blue', lw=2, label='Total Loss (Avg)')
ax1.tick_params(axis='y', labelcolor='tab:blue')
ax1.grid(True, linestyle='--', alpha=0.5)

ax2 = ax1.twinx()
ax2.set_ylabel('Physics Loss ($L_{phys}$)', color='tab:red')
ax2.plot(epochs_range, avg_phys_loss, color='tab:red', lw=2, linestyle=':', label='Physics Loss (Avg)')
ax2.tick_params(axis='y', labelcolor='tab:red')

plt.title(f'PIGNN Learning Curve (CV Average, Alpha={FIXED_ALPHA})')
fig.tight_layout()
plt.show()

In [ ]:
# ==========================================
# 2. メイン実行セクション (CVループ) - 学習曲線記録版
# ==========================================
import matplotlib.pyplot as plt # 追加

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 論文同様、固定値で評価（0, 1.0, 10.0など）
FIXED_ALPHA = 10.0

v16_load_path = "/content/drive/MyDrive/GNN_Football_Analysis/Processed_Data/gnn_data_v17_final.pt"
print(f"CV用マスターデータをロード中: {v16_load_path}")
checkpoint = torch.load(v16_load_path, weights_only=False)
all_data_list = checkpoint['all_data']

# --- 【修正】推測ロジックを削除し、刻印されたIDを直接取得 ---
match_ids = sorted(list(set([int(d.match_id.item()) for d in all_data_list])))
print(f"検出された試合ID: {match_ids} (計 {len(match_ids)} 試合)")

# ハイパーパラメータ
EPOCHS_CV =100
LR = 0.0005
cv_final_reports = []
best_overall_f1 = 0

# --- 【追加】全CVの学習ログを保存する箱 ---
all_cv_history = []

print(f"PIGNN クロスバリデーション開始 (Alpha={FIXED_ALPHA} 固定モード)\n")

for test_match in match_ids:
    print(f"\n{'='*60}")
    print(f" 🌀 Round: Match {test_match} をテストに使用")
    print(f"{'='*60}")

    # 1. 刻印された match_id を信じて切り分け
    test_indices = [d for d in all_data_list if int(d.match_id.item()) == test_match]
    train_candidates = [d for d in all_data_list if int(d.match_id.item()) != test_match]

    # 2. 訓練データのみ1:1アンダーサンプリング
    cv_train_set = balance_dataset_by_undersampling(train_candidates)

    cv_train_loader = DataLoader(cv_train_set, batch_size=32, shuffle=True)
    cv_test_loader = DataLoader(test_indices, batch_size=32, shuffle=False)

    cv_model = PIGNNClassifier(hidden_channels=64).to(device)
    cv_optimizer = torch.optim.Adam(cv_model.parameters(), lr=LR)

    # --- 【追加】このRoundのログを保持する辞書 ---
    round_history = {'total_loss': [], 'physics_loss': []}

    # 3. 学習ループ (固定Alpha版)
    for epoch in range(1, EPOCHS_CV + 1):
        loss, phys, _ = train_pignn_epoch_fixed(cv_model, cv_train_loader, cv_optimizer, device, FIXED_ALPHA)

        # --- 【追加】ログを記録 ---
        round_history['total_loss'].append(loss)
        round_history['physics_loss'].append(phys)

        if epoch % 10 == 0 or epoch == 1:
            print(f"  Epoch {epoch:02d} | Loss: {loss:.4f} | Phys_L: {phys:.6f}")

    all_cv_history.append(round_history) # 追加

    # 4. 評価
    cv_model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for data in cv_test_loader:
            data = data.to(device)
            out = cv_model(data)
            y_true.extend(data.y.view(-1).cpu().numpy())
            y_pred.extend(out.argmax(dim=1).cpu().numpy())

    # スコア集計
    report = classification_report(y_true, y_pred, output_dict=True, zero_division=0)
    current_f1 = report['1']['f1-score']

    cv_final_reports.append({
        'match': test_match,
        'recall': report['1']['recall'],
        'precision': report['1']['precision'],
        'f1': current_f1
    })

    # --- α別にフォルダを分けて整理保存 ---
    import os
    base_model_dir = "/content/drive/MyDrive/GNN_Football_Analysis/Models"
    alpha_dir = os.path.join(base_model_dir, f"alpha_{str(FIXED_ALPHA).replace('.', '_')}_take3")
    os.makedirs(alpha_dir, exist_ok=True)

    model_filename = f'pignn_testmatch_{test_match}.pth'
    model_path = os.path.join(alpha_dir, model_filename)
    torch.save(cv_model.state_dict(), model_path)
    print(f" >> [Alpha {FIXED_ALPHA}] Match {test_match} weight saved.")

    if current_f1 > best_overall_f1:
        best_overall_f1 = current_f1
        best_model_path = os.path.join(alpha_dir, f'best_overall_alpha_{FIXED_ALPHA}.pth')
        torch.save(cv_model.state_dict(), best_model_path)
        print(f"  [Alpha {FIXED_ALPHA}] New Best Model Saved: F1={best_overall_f1:.4f}")

    print(f" >> Result: Recall={report['1']['recall']:.4f}, Precision={report['1']['precision']:.4f}, F1={current_f1:.4f}")

# 5. 最終集計
print(f"\n\n{'#'*60}\n  Alpha={FIXED_ALPHA} CV 最終平均結果\n{'#'*60}")
avg_recall = np.mean([r['recall'] for r in cv_final_reports])
avg_f1 = np.mean([r['f1'] for r in cv_final_reports])
avg_precision = np.mean([r['precision'] for r in cv_final_reports])

print(f"\n[OVERALL] Avg Success Recall:     {avg_recall:.4f}")
print(f"[OVERALL] Avg Success Precision: {avg_precision:.4f}")
print(f"[OVERALL] Avg Success F1-score:  {avg_f1:.4f}")

# ==========================================
# 6. 学習曲線の描画 (CV平均) - 【追加セクション】
# ==========================================
avg_total_loss = np.mean([h['total_loss'] for h in all_cv_history], axis=0)
avg_phys_loss = np.mean([h['physics_loss'] for h in all_cv_history], axis=0)
epochs_range = range(1, EPOCHS_CV + 1)

fig, ax1 = plt.subplots(figsize=(10, 6))

ax1.set_xlabel('Epochs')
ax1.set_ylabel('Total Training Loss', color='tab:blue')
ax1.plot(epochs_range, avg_total_loss, color='tab:blue', lw=2, label='Total Loss (Avg)')
ax1.tick_params(axis='y', labelcolor='tab:blue')
ax1.grid(True, linestyle='--', alpha=0.5)

ax2 = ax1.twinx()
ax2.set_ylabel('Physics Loss ($L_{phys}$)', color='tab:red')
ax2.plot(epochs_range, avg_phys_loss, color='tab:red', lw=2, linestyle=':', label='Physics Loss (Avg)')
ax2.tick_params(axis='y', labelcolor='tab:red')

plt.title(f'PIGNN Learning Curve (CV Average, Alpha={FIXED_ALPHA})')
fig.tight_layout()
plt.show()

In [ ]:
# ==========================================
# 2. メイン実行セクション (CVループ) - 学習曲線記録版
# ==========================================
import matplotlib.pyplot as plt # 追加

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 論文同様、固定値で評価（0, 1.0, 10.0など）
FIXED_ALPHA = 100.0
v16_load_path = "/content/drive/MyDrive/GNN_Football_Analysis/Processed_Data/gnn_data_v17_final.pt"
print(f"CV用マスターデータをロード中: {v16_load_path}")
checkpoint = torch.load(v16_load_path, weights_only=False)
all_data_list = checkpoint['all_data']

# --- 【修正】推測ロジックを削除し、刻印されたIDを直接取得 ---
match_ids = sorted(list(set([int(d.match_id.item()) for d in all_data_list])))
print(f"検出された試合ID: {match_ids} (計 {len(match_ids)} 試合)")

# ハイパーパラメータ
EPOCHS_CV =100
LR = 0.0005
cv_final_reports = []
best_overall_f1 = 0

# --- 【追加】全CVの学習ログを保存する箱 ---
all_cv_history = []

print(f"PIGNN クロスバリデーション開始 (Alpha={FIXED_ALPHA} 固定モード)\n")

for test_match in match_ids:
    print(f"\n{'='*60}")
    print(f"  Round: Match {test_match} をテストに使用")
    print(f"{'='*60}")

    # 1. 刻印された match_id を信じて切り分け
    test_indices = [d for d in all_data_list if int(d.match_id.item()) == test_match]
    train_candidates = [d for d in all_data_list if int(d.match_id.item()) != test_match]

    # 2. 訓練データのみ1:1アンダーサンプリング
    cv_train_set = balance_dataset_by_undersampling(train_candidates)

    cv_train_loader = DataLoader(cv_train_set, batch_size=32, shuffle=True)
    cv_test_loader = DataLoader(test_indices, batch_size=32, shuffle=False)

    cv_model = PIGNNClassifier(hidden_channels=64).to(device)
    cv_optimizer = torch.optim.Adam(cv_model.parameters(), lr=LR)

    # --- 【追加】このRoundのログを保持する辞書 ---
    round_history = {'total_loss': [], 'physics_loss': []}

    # 3. 学習ループ (固定Alpha版)
    for epoch in range(1, EPOCHS_CV + 1):
        loss, phys, _ = train_pignn_epoch_fixed(cv_model, cv_train_loader, cv_optimizer, device, FIXED_ALPHA)

        # --- 【追加】ログを記録 ---
        round_history['total_loss'].append(loss)
        round_history['physics_loss'].append(phys)

        if epoch % 10 == 0 or epoch == 1:
            print(f"  Epoch {epoch:02d} | Loss: {loss:.4f} | Phys_L: {phys:.6f}")

    all_cv_history.append(round_history) # 追加

    # 4. 評価
    cv_model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for data in cv_test_loader:
            data = data.to(device)
            out = cv_model(data)
            y_true.extend(data.y.view(-1).cpu().numpy())
            y_pred.extend(out.argmax(dim=1).cpu().numpy())

    # スコア集計
    report = classification_report(y_true, y_pred, output_dict=True, zero_division=0)
    current_f1 = report['1']['f1-score']

    cv_final_reports.append({
        'match': test_match,
        'recall': report['1']['recall'],
        'precision': report['1']['precision'],
        'f1': current_f1
    })

    # --- α別にフォルダを分けて整理保存 ---
    import os
    base_model_dir = "/content/drive/MyDrive/GNN_Football_Analysis/Models"
    alpha_dir = os.path.join(base_model_dir, f"alpha_{str(FIXED_ALPHA).replace('.', '_')}_take3")
    os.makedirs(alpha_dir, exist_ok=True)

    model_filename = f'pignn_testmatch_{test_match}.pth'
    model_path = os.path.join(alpha_dir, model_filename)
    torch.save(cv_model.state_dict(), model_path)
    print(f" >> [Alpha {FIXED_ALPHA}] Match {test_match} weight saved.")

    if current_f1 > best_overall_f1:
        best_overall_f1 = current_f1
        best_model_path = os.path.join(alpha_dir, f'best_overall_alpha_{FIXED_ALPHA}.pth')
        torch.save(cv_model.state_dict(), best_model_path)
        print(f"  [Alpha {FIXED_ALPHA}] New Best Model Saved: F1={best_overall_f1:.4f}")

    print(f" >> Result: Recall={report['1']['recall']:.4f}, Precision={report['1']['precision']:.4f}, F1={current_f1:.4f}")

# 5. 最終集計
print(f"\n\n{'#'*60}\n  Alpha={FIXED_ALPHA} CV 最終平均結果\n{'#'*60}")
avg_recall = np.mean([r['recall'] for r in cv_final_reports])
avg_f1 = np.mean([r['f1'] for r in cv_final_reports])
avg_precision = np.mean([r['precision'] for r in cv_final_reports])

print(f"\n[OVERALL] Avg Success Recall:     {avg_recall:.4f}")
print(f"[OVERALL] Avg Success Precision: {avg_precision:.4f}")
print(f"[OVERALL] Avg Success F1-score:  {avg_f1:.4f}")

# ==========================================
# 6. 学習曲線の描画 (CV平均) - 【追加セクション】
# ==========================================
avg_total_loss = np.mean([h['total_loss'] for h in all_cv_history], axis=0)
avg_phys_loss = np.mean([h['physics_loss'] for h in all_cv_history], axis=0)
epochs_range = range(1, EPOCHS_CV + 1)

fig, ax1 = plt.subplots(figsize=(10, 6))

ax1.set_xlabel('Epochs')
ax1.set_ylabel('Total Training Loss', color='tab:blue')
ax1.plot(epochs_range, avg_total_loss, color='tab:blue', lw=2, label='Total Loss (Avg)')
ax1.tick_params(axis='y', labelcolor='tab:blue')
ax1.grid(True, linestyle='--', alpha=0.5)

ax2 = ax1.twinx()
ax2.set_ylabel('Physics Loss ($L_{phys}$)', color='tab:red')
ax2.plot(epochs_range, avg_phys_loss, color='tab:red', lw=2, linestyle=':', label='Physics Loss (Avg)')
ax2.tick_params(axis='y', labelcolor='tab:red')

plt.title(f'PIGNN Learning Curve (CV Average, Alpha={FIXED_ALPHA})')
fig.tight_layout()
plt.show()

In [ ]:
# ==========================================
# 2. メイン実行セクション (CVループ) - 学習曲線記録版
# ==========================================
import matplotlib.pyplot as plt # 追加

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 論文同様、固定値で評価（0, 1.0, 10.0など）
FIXED_ALPHA = 200.0
v16_load_path = "/content/drive/MyDrive/GNN_Football_Analysis/Processed_Data/gnn_data_v17_final.pt"
print(f"CV用マスターデータをロード中: {v16_load_path}")
checkpoint = torch.load(v16_load_path, weights_only=False)
all_data_list = checkpoint['all_data']

# --- 【修正】推測ロジックを削除し、刻印されたIDを直接取得 ---
match_ids = sorted(list(set([int(d.match_id.item()) for d in all_data_list])))
print(f"検出された試合ID: {match_ids} (計 {len(match_ids)} 試合)")

# ハイパーパラメータ
EPOCHS_CV =100
LR = 0.0005
cv_final_reports = []
best_overall_f1 = 0

# --- 【追加】全CVの学習ログを保存する箱 ---
all_cv_history = []

print(f"PIGNN クロスバリデーション開始 (Alpha={FIXED_ALPHA} 固定モード)\n")

for test_match in match_ids:
    print(f"\n{'='*60}")
    print(f"  Round: Match {test_match} をテストに使用")
    print(f"{'='*60}")

    # 1. 刻印された match_id を信じて切り分け
    test_indices = [d for d in all_data_list if int(d.match_id.item()) == test_match]
    train_candidates = [d for d in all_data_list if int(d.match_id.item()) != test_match]

    # 2. 訓練データのみ1:1アンダーサンプリング
    cv_train_set = balance_dataset_by_undersampling(train_candidates)

    cv_train_loader = DataLoader(cv_train_set, batch_size=32, shuffle=True)
    cv_test_loader = DataLoader(test_indices, batch_size=32, shuffle=False)

    cv_model = PIGNNClassifier(hidden_channels=64).to(device)
    cv_optimizer = torch.optim.Adam(cv_model.parameters(), lr=LR)

    # --- 【追加】このRoundのログを保持する辞書 ---
    round_history = {'total_loss': [], 'physics_loss': []}

    # 3. 学習ループ (固定Alpha版)
    for epoch in range(1, EPOCHS_CV + 1):
        loss, phys, _ = train_pignn_epoch_fixed(cv_model, cv_train_loader, cv_optimizer, device, FIXED_ALPHA)

        # --- 【追加】ログを記録 ---
        round_history['total_loss'].append(loss)
        round_history['physics_loss'].append(phys)

        if epoch % 10 == 0 or epoch == 1:
            print(f"  Epoch {epoch:02d} | Loss: {loss:.4f} | Phys_L: {phys:.6f}")

    all_cv_history.append(round_history) # 追加

    # 4. 評価
    cv_model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for data in cv_test_loader:
            data = data.to(device)
            out = cv_model(data)
            y_true.extend(data.y.view(-1).cpu().numpy())
            y_pred.extend(out.argmax(dim=1).cpu().numpy())

    # スコア集計
    report = classification_report(y_true, y_pred, output_dict=True, zero_division=0)
    current_f1 = report['1']['f1-score']

    cv_final_reports.append({
        'match': test_match,
        'recall': report['1']['recall'],
        'precision': report['1']['precision'],
        'f1': current_f1
    })

    # --- α別にフォルダを分けて整理保存 ---
    import os
    base_model_dir = "/content/drive/MyDrive/GNN_Football_Analysis/Models"
    alpha_dir = os.path.join(base_model_dir, f"alpha_{str(FIXED_ALPHA).replace('.', '_')}_take3")
    os.makedirs(alpha_dir, exist_ok=True)

    model_filename = f'pignn_testmatch_{test_match}.pth'
    model_path = os.path.join(alpha_dir, model_filename)
    torch.save(cv_model.state_dict(), model_path)
    print(f" >> [Alpha {FIXED_ALPHA}] Match {test_match} weight saved.")

    if current_f1 > best_overall_f1:
        best_overall_f1 = current_f1
        best_model_path = os.path.join(alpha_dir, f'best_overall_alpha_{FIXED_ALPHA}.pth')
        torch.save(cv_model.state_dict(), best_model_path)
        print(f"  [Alpha {FIXED_ALPHA}] New Best Model Saved: F1={best_overall_f1:.4f}")

    print(f" >> Result: Recall={report['1']['recall']:.4f}, Precision={report['1']['precision']:.4f}, F1={current_f1:.4f}")

# 5. 最終集計
print(f"\n\n{'#'*60}\n  Alpha={FIXED_ALPHA} CV 最終平均結果\n{'#'*60}")
avg_recall = np.mean([r['recall'] for r in cv_final_reports])
avg_f1 = np.mean([r['f1'] for r in cv_final_reports])
avg_precision = np.mean([r['precision'] for r in cv_final_reports])

print(f"\n[OVERALL] Avg Success Recall:     {avg_recall:.4f}")
print(f"[OVERALL] Avg Success Precision: {avg_precision:.4f}")
print(f"[OVERALL] Avg Success F1-score:  {avg_f1:.4f}")

# ==========================================
# 6. 学習曲線の描画 (CV平均) - 【追加セクション】
# ==========================================
avg_total_loss = np.mean([h['total_loss'] for h in all_cv_history], axis=0)
avg_phys_loss = np.mean([h['physics_loss'] for h in all_cv_history], axis=0)
epochs_range = range(1, EPOCHS_CV + 1)

fig, ax1 = plt.subplots(figsize=(10, 6))

ax1.set_xlabel('Epochs')
ax1.set_ylabel('Total Training Loss', color='tab:blue')
ax1.plot(epochs_range, avg_total_loss, color='tab:blue', lw=2, label='Total Loss (Avg)')
ax1.tick_params(axis='y', labelcolor='tab:blue')
ax1.grid(True, linestyle='--', alpha=0.5)

ax2 = ax1.twinx()
ax2.set_ylabel('Physics Loss ($L_{phys}$)', color='tab:red')
ax2.plot(epochs_range, avg_phys_loss, color='tab:red', lw=2, linestyle=':', label='Physics Loss (Avg)')
ax2.tick_params(axis='y', labelcolor='tab:red')

plt.title(f'PIGNN Learning Curve (CV Average, Alpha={FIXED_ALPHA})')
fig.tight_layout()
plt.show()

In [ ]:
import os
import torch
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

# ==========================================
# 設定：検証したい Alpha を指定
# ==========================================
FIXED_ALPHA = 0  # 比較のためにここを 0 や 1.0 に切り替えて実行
base_model_dir = "/content/drive/MyDrive/GNN_Football_Analysis/Models"
# _take2 を含める場合
alpha_folder = f"alpha_{str(FIXED_ALPHA).replace('.', '_')}_take3"
model_load_dir = os.path.join(base_model_dir, alpha_folder)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

all_preds = []
all_labels = []
success_team_vxs = []

print(f" Alpha={FIXED_ALPHA} の全試合モデルを統合評価中...")

# --- CVの結果を再現するために、各試合のモデルを個別にロードしてテスト ---
# match_ids は CV実行時と同じ [1, 2, 3, 4, 5, 6, 7]
for test_match in match_ids:
    # 1. その試合用のデータを抽出
    test_indices = [d for d in all_data_list if int(d.match_id.item()) == test_match]
    test_loader = DataLoader(test_indices, batch_size=32, shuffle=False)

    # 2. その試合で訓練されたモデル重みをロード
    model_path = os.path.join(model_load_dir, f'pignn_testmatch_{test_match}.pth')
    if not os.path.exists(model_path):
        print(f" Skip: {model_path} が見つかりません")
        continue

    model = PIGNNClassifier(hidden_channels=64).to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    # 3. 予測と物理情報の抽出
    with torch.no_grad():
        for data in test_loader:
            data = data.to(device)
            out = model(data)
            pred = out.argmax(dim=1)

            # 各シーン(グラフ)ごとの物理量抽出
            for i in range(out.size(0)):
                mask = (data.batch == i)
                # 論文(cite: 12)のPermutation Importanceで最重要視されたvxを計算
                avg_vx = torch.mean(data.vel[mask, 0]).item()

                if pred[i] == 1: # AIが成功(1)と予測した時のみ
                    success_team_vxs.append(avg_vx)

            all_preds.extend(pred.cpu().numpy())
            all_labels.extend(data.y.view(-1).cpu().numpy())

# ==========================================
# 4. レポート表示セクション
# ==========================================
print("\n" + "="*60)
print(f"  PIGNN 最終クラシフィケーションレポート (Alpha={FIXED_ALPHA})")
print("="*60)
# 論文(cite: 115)の「naïve baseline」との比較を念頭に置いた出力
print(classification_report(all_labels, all_preds, target_names=['Failure', 'Success'], zero_division=0))

print("\n" + "="*60)
print(f"  物理的整合性 検証 (vx = Byline to Byline Speed)")
print("="*60)
if len(success_team_vxs) > 0:
    avg_team_vx = np.mean(success_team_vxs)
    # 論文(cite: 196)で「highest impact」とされたvxの方向性を確認
    positive_ratio = np.sum(np.array(success_team_vxs) > 0) / len(success_team_vxs)

    print(f"成功予測シーンの平均 vx: {avg_team_vx:.4f} m/s")
    print(f"右向き(正方向)への推進力割合: {positive_ratio*100:.1f} %")

    # 論文(cite: 6, 33)の「high speed attack」の定義に基づき評価
    if positive_ratio > 0.65:
        print(">> 判定: 物理的妥当。モデルは論文の定義通り『前方への速度』を重視しています。")
    else:
        print(">> 判定: 物理的乖離あり。戦術的特徴よりもノイズを学習している可能性があります。")
else:
    print("Successと予測されたフレームがありませんでした。")

# 5. 混同行列の描画
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens' if FIXED_ALPHA > 0 else 'Blues',
            xticklabels=['Fail', 'Success'], yticklabels=['Fail', 'Success'])
plt.title(f'Confusion Matrix (Alpha={FIXED_ALPHA})')
plt.show()

In [ ]:
import os
import torch
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

# ==========================================
# 設定：検証したい Alpha を指定
# ==========================================
FIXED_ALPHA = 50.0  # 比較のためにここを 0 や 1.0 に切り替えて実行
base_model_dir = "/content/drive/MyDrive/GNN_Football_Analysis/Models"
# _take2 を含める場合
alpha_folder = f"alpha_{str(FIXED_ALPHA).replace('.', '_')}_take3"
model_load_dir = os.path.join(base_model_dir, alpha_folder)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

all_preds = []
all_labels = []
success_team_vxs = []

print(f" Alpha={FIXED_ALPHA} の全試合モデルを統合評価中...")

# --- CVの結果を再現するために、各試合のモデルを個別にロードしてテスト ---
# match_ids は CV実行時と同じ [1, 2, 3, 4, 5, 6, 7]
for test_match in match_ids:
    # 1. その試合用のデータを抽出
    test_indices = [d for d in all_data_list if int(d.match_id.item()) == test_match]
    test_loader = DataLoader(test_indices, batch_size=32, shuffle=False)

    # 2. その試合で訓練されたモデル重みをロード
    model_path = os.path.join(model_load_dir, f'pignn_testmatch_{test_match}.pth')
    if not os.path.exists(model_path):
        print(f" Skip: {model_path} が見つかりません")
        continue

    model = PIGNNClassifier(hidden_channels=64).to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    # 3. 予測と物理情報の抽出
    with torch.no_grad():
        for data in test_loader:
            data = data.to(device)
            out = model(data)
            pred = out.argmax(dim=1)

            # 各シーン(グラフ)ごとの物理量抽出
            for i in range(out.size(0)):
                mask = (data.batch == i)
                # 論文(cite: 12)のPermutation Importanceで最重要視されたvxを計算
                avg_vx = torch.mean(data.vel[mask, 0]).item()

                if pred[i] == 1: # AIが成功(1)と予測した時のみ
                    success_team_vxs.append(avg_vx)

            all_preds.extend(pred.cpu().numpy())
            all_labels.extend(data.y.view(-1).cpu().numpy())

# ==========================================
# 4. レポート表示セクション
# ==========================================
print("\n" + "="*60)
print(f"  PIGNN 最終クラシフィケーションレポート (Alpha={FIXED_ALPHA})")
print("="*60)
# 論文(cite: 115)の「naïve baseline」との比較を念頭に置いた出力
print(classification_report(all_labels, all_preds, target_names=['Failure', 'Success'], zero_division=0))

print("\n" + "="*60)
print(f"  物理的整合性 検証 (vx = Byline to Byline Speed)")
print("="*60)
if len(success_team_vxs) > 0:
    avg_team_vx = np.mean(success_team_vxs)
    # 論文(cite: 196)で「highest impact」とされたvxの方向性を確認
    positive_ratio = np.sum(np.array(success_team_vxs) > 0) / len(success_team_vxs)

    print(f"成功予測シーンの平均 vx: {avg_team_vx:.4f} m/s")
    print(f"右向き(正方向)への推進力割合: {positive_ratio*100:.1f} %")

    # 論文(cite: 6, 33)の「high speed attack」の定義に基づき評価
    if positive_ratio > 0.65:
        print(">> 判定: 物理的妥当。モデルは論文の定義通り『前方への速度』を重視しています。")
    else:
        print(">> 判定: 物理的乖離あり。戦術的特徴よりもノイズを学習している可能性があります。")
else:
    print("Successと予測されたフレームがありませんでした。")

# 5. 混同行列の描画
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens' if FIXED_ALPHA > 0 else 'Blues',
            xticklabels=['Fail', 'Success'], yticklabels=['Fail', 'Success'])
plt.title(f'Confusion Matrix (Alpha={FIXED_ALPHA})')
plt.show()

In [ ]:
import os
import torch
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

# ==========================================
# 設定：検証したい Alpha を指定
# ==========================================
FIXED_ALPHA = 10.0  # 比較のためにここを 0 や 1.0 に切り替えて実行
base_model_dir = "/content/drive/MyDrive/GNN_Football_Analysis/Models"
# _take2 を含める場合
alpha_folder = f"alpha_{str(FIXED_ALPHA).replace('.', '_')}_take3"
model_load_dir = os.path.join(base_model_dir, alpha_folder)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

all_preds = []
all_labels = []
success_team_vxs = []

print(f" Alpha={FIXED_ALPHA} の全試合モデルを統合評価中...")

# --- CVの結果を再現するために、各試合のモデルを個別にロードしてテスト ---
# match_ids は CV実行時と同じ [1, 2, 3, 4, 5, 6, 7]
for test_match in match_ids:
    # 1. その試合用のデータを抽出
    test_indices = [d for d in all_data_list if int(d.match_id.item()) == test_match]
    test_loader = DataLoader(test_indices, batch_size=32, shuffle=False)

    # 2. その試合で訓練されたモデル重みをロード
    model_path = os.path.join(model_load_dir, f'pignn_testmatch_{test_match}.pth')
    if not os.path.exists(model_path):
        print(f" Skip: {model_path} が見つかりません")
        continue

    model = PIGNNClassifier(hidden_channels=64).to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    # 3. 予測と物理情報の抽出
    with torch.no_grad():
        for data in test_loader:
            data = data.to(device)
            out = model(data)
            pred = out.argmax(dim=1)

            # 各シーン(グラフ)ごとの物理量抽出
            for i in range(out.size(0)):
                mask = (data.batch == i)
                # 論文(cite: 12)のPermutation Importanceで最重要視されたvxを計算
                avg_vx = torch.mean(data.vel[mask, 0]).item()

                if pred[i] == 1: # AIが成功(1)と予測した時のみ
                    success_team_vxs.append(avg_vx)

            all_preds.extend(pred.cpu().numpy())
            all_labels.extend(data.y.view(-1).cpu().numpy())

# ==========================================
# 4. レポート表示セクション
# ==========================================
print("\n" + "="*60)
print(f"  PIGNN 最終クラシフィケーションレポート (Alpha={FIXED_ALPHA})")
print("="*60)
# 論文(cite: 115)の「naïve baseline」との比較を念頭に置いた出力
print(classification_report(all_labels, all_preds, target_names=['Failure', 'Success'], zero_division=0))

print("\n" + "="*60)
print(f"  物理的整合性 検証 (vx = Byline to Byline Speed)")
print("="*60)
if len(success_team_vxs) > 0:
    avg_team_vx = np.mean(success_team_vxs)
    # 論文(cite: 196)で「highest impact」とされたvxの方向性を確認
    positive_ratio = np.sum(np.array(success_team_vxs) > 0) / len(success_team_vxs)

    print(f"成功予測シーンの平均 vx: {avg_team_vx:.4f} m/s")
    print(f"右向き(正方向)への推進力割合: {positive_ratio*100:.1f} %")

    # 論文(cite: 6, 33)の「high speed attack」の定義に基づき評価
    if positive_ratio > 0.65:
        print(">> 判定: 物理的妥当。モデルは論文の定義通り『前方への速度』を重視しています。")
    else:
        print(">> 判定: 物理的乖離あり。戦術的特徴よりもノイズを学習している可能性があります。")
else:
    print("Successと予測されたフレームがありませんでした。")

# 5. 混同行列の描画
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens' if FIXED_ALPHA > 0 else 'Blues',
            xticklabels=['Fail', 'Success'], yticklabels=['Fail', 'Success'])
plt.title(f'Confusion Matrix (Alpha={FIXED_ALPHA})')
plt.show()

In [ ]:
import os
import torch
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

# ==========================================
# 設定：検証したい Alpha を指定
# ==========================================
FIXED_ALPHA = 100.0  # 比較のためにここを 0 や 1.0 に切り替えて実行
base_model_dir = "/content/drive/MyDrive/GNN_Football_Analysis/Models"
# _take2 を含める場合
alpha_folder = f"alpha_{str(FIXED_ALPHA).replace('.', '_')}_take3"
model_load_dir = os.path.join(base_model_dir, alpha_folder)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

all_preds = []
all_labels = []
success_team_vxs = []

print(f" Alpha={FIXED_ALPHA} の全試合モデルを統合評価中...")

# --- CVの結果を再現するために、各試合のモデルを個別にロードしてテスト ---
# match_ids は CV実行時と同じ [1, 2, 3, 4, 5, 6, 7]
for test_match in match_ids:
    # 1. その試合用のデータを抽出
    test_indices = [d for d in all_data_list if int(d.match_id.item()) == test_match]
    test_loader = DataLoader(test_indices, batch_size=32, shuffle=False)

    # 2. その試合で訓練されたモデル重みをロード
    model_path = os.path.join(model_load_dir, f'pignn_testmatch_{test_match}.pth')
    if not os.path.exists(model_path):
        print(f" Skip: {model_path} が見つかりません")
        continue

    model = PIGNNClassifier(hidden_channels=64).to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    # 3. 予測と物理情報の抽出
    with torch.no_grad():
        for data in test_loader:
            data = data.to(device)
            out = model(data)
            pred = out.argmax(dim=1)

            # 各シーン(グラフ)ごとの物理量抽出
            for i in range(out.size(0)):
                mask = (data.batch == i)
                # 論文(cite: 12)のPermutation Importanceで最重要視されたvxを計算
                avg_vx = torch.mean(data.vel[mask, 0]).item()

                if pred[i] == 1: # AIが成功(1)と予測した時のみ
                    success_team_vxs.append(avg_vx)

            all_preds.extend(pred.cpu().numpy())
            all_labels.extend(data.y.view(-1).cpu().numpy())

# ==========================================
# 4. レポート表示セクション
# ==========================================
print("\n" + "="*60)
print(f"  PIGNN 最終クラシフィケーションレポート (Alpha={FIXED_ALPHA})")
print("="*60)
# 論文(cite: 115)の「naïve baseline」との比較を念頭に置いた出力
print(classification_report(all_labels, all_preds, target_names=['Failure', 'Success'], zero_division=0))

print("\n" + "="*60)
print(f"  物理的整合性 検証 (vx = Byline to Byline Speed)")
print("="*60)
if len(success_team_vxs) > 0:
    avg_team_vx = np.mean(success_team_vxs)
    # 論文(cite: 196)で「highest impact」とされたvxの方向性を確認
    positive_ratio = np.sum(np.array(success_team_vxs) > 0) / len(success_team_vxs)

    print(f"成功予測シーンの平均 vx: {avg_team_vx:.4f} m/s")
    print(f"右向き(正方向)への推進力割合: {positive_ratio*100:.1f} %")

    # 論文(cite: 6, 33)の「high speed attack」の定義に基づき評価
    if positive_ratio > 0.65:
        print(">> 判定: 物理的妥当。モデルは論文の定義通り『前方への速度』を重視しています。")
    else:
        print(">> 判定: 物理的乖離あり。戦術的特徴よりもノイズを学習している可能性があります。")
else:
    print("Successと予測されたフレームがありませんでした。")

# 5. 混同行列の描画
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens' if FIXED_ALPHA > 0 else 'Blues',
            xticklabels=['Fail', 'Success'], yticklabels=['Fail', 'Success'])
plt.title(f'Confusion Matrix (Alpha={FIXED_ALPHA})')
plt.show()

In [ ]:
import os
import torch
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

# ==========================================
# 設定：検証したい Alpha を指定
# ==========================================
FIXED_ALPHA = 200.0  # 比較のためにここを 0 や 1.0 に切り替えて実行
base_model_dir = "/content/drive/MyDrive/GNN_Football_Analysis/Models"
# _take2 を含める場合
alpha_folder = f"alpha_{str(FIXED_ALPHA).replace('.', '_')}_take3"
model_load_dir = os.path.join(base_model_dir, alpha_folder)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

all_preds = []
all_labels = []
success_team_vxs = []

print(f" Alpha={FIXED_ALPHA} の全試合モデルを統合評価中...")

# --- CVの結果を再現するために、各試合のモデルを個別にロードしてテスト ---
# match_ids は CV実行時と同じ [1, 2, 3, 4, 5, 6, 7]
for test_match in match_ids:
    # 1. その試合用のデータを抽出
    test_indices = [d for d in all_data_list if int(d.match_id.item()) == test_match]
    test_loader = DataLoader(test_indices, batch_size=32, shuffle=False)

    # 2. その試合で訓練されたモデル重みをロード
    model_path = os.path.join(model_load_dir, f'pignn_testmatch_{test_match}.pth')
    if not os.path.exists(model_path):
        print(f" Skip: {model_path} が見つかりません")
        continue

    model = PIGNNClassifier(hidden_channels=64).to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    # 3. 予測と物理情報の抽出
    with torch.no_grad():
        for data in test_loader:
            data = data.to(device)
            out = model(data)
            pred = out.argmax(dim=1)

            # 各シーン(グラフ)ごとの物理量抽出
            for i in range(out.size(0)):
                mask = (data.batch == i)
                # 論文(cite: 12)のPermutation Importanceで最重要視されたvxを計算
                avg_vx = torch.mean(data.vel[mask, 0]).item()

                if pred[i] == 1: # AIが成功(1)と予測した時のみ
                    success_team_vxs.append(avg_vx)

            all_preds.extend(pred.cpu().numpy())
            all_labels.extend(data.y.view(-1).cpu().numpy())

# ==========================================
# 4. レポート表示セクション
# ==========================================
print("\n" + "="*60)
print(f"  PIGNN 最終クラシフィケーションレポート (Alpha={FIXED_ALPHA})")
print("="*60)
# 論文(cite: 115)の「naïve baseline」との比較を念頭に置いた出力
print(classification_report(all_labels, all_preds, target_names=['Failure', 'Success'], zero_division=0))

print("\n" + "="*60)
print(f"  物理的整合性 検証 (vx = Byline to Byline Speed)")
print("="*60)
if len(success_team_vxs) > 0:
    avg_team_vx = np.mean(success_team_vxs)
    # 論文(cite: 196)で「highest impact」とされたvxの方向性を確認
    positive_ratio = np.sum(np.array(success_team_vxs) > 0) / len(success_team_vxs)

    print(f"成功予測シーンの平均 vx: {avg_team_vx:.4f} m/s")
    print(f"右向き(正方向)への推進力割合: {positive_ratio*100:.1f} %")

    # 論文(cite: 6, 33)の「high speed attack」の定義に基づき評価
    if positive_ratio > 0.65:
        print(">> 判定: 物理的妥当。モデルは論文の定義通り『前方への速度』を重視しています。")
    else:
        print(">> 判定: 物理的乖離あり。戦術的特徴よりもノイズを学習している可能性があります。")
else:
    print("Successと予測されたフレームがありませんでした。")

# 5. 混同行列の描画
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens' if FIXED_ALPHA > 0 else 'Blues',
            xticklabels=['Fail', 'Success'], yticklabels=['Fail', 'Success'])
plt.title(f'Confusion Matrix (Alpha={FIXED_ALPHA})')
plt.show()

In [ ]:
import torch
from torch_geometric.data import Data

def run_leakage_diagnostic(model, loader, device):
    model.eval()
    # 特徴量ラベル: 0:x, 1:y, 2:vx, 3:vy, 4:dist_goal, 5:dist_ball, 6:team_id
    feature_names = ["x", "y", "vx", "vy", "dist_goal", "dist_ball", "team_id"]

    print(f"{'Removed Feature':<15} | {'Test Acc':<10} | {'Recall (S)':<10}")
    print("-" * 45)

    # ベースライン（全特徴量あり）
    base_acc = test_pignn(model, loader, device)
    print(f"{'None (Baseline)':<15} | {base_acc:.4f}")

    with torch.no_grad():
        for i in range(7):
            correct = 0
            tp = 0 # True Positive
            fn = 0 # False Negative

            for data in loader:
                data = preprocess_batch(data, device)

                # 特定の特徴量をゼロに置き換える
                x_shuffled = data.x.clone()
                x_shuffled[:, i] = 0.0

                # 推論
                out = model(Data(x=x_shuffled, edge_index=data.edge_index,
                                 batch=data.batch, pos=data.pos, vel=data.vel))
                pred = out.argmax(dim=1)

                # 精度計算
                y_true = data.y.view(-1)
                correct += (pred == y_true).sum().item()

                # Recall (Success) 計算
                tp += ((pred == 1) & (y_true == 1)).sum().item()
                fn += ((pred == 0) & (y_true == 1)).sum().item()

            acc = correct / len(loader.dataset)
            recall = tp / (tp + fn) if (tp + fn) > 0 else 0
            print(f"{feature_names[i]:<15} | {acc:.4f}     | {recall:.4f}")

# 実行
run_leakage_diagnostic(model, test_loader, device)

ただのMLP

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import global_mean_pool
from torch_geometric.loader import DataLoader
from sklearn.metrics import classification_report
import os
import numpy as np

# ==========================================
# 1. MLPモデルの定義 (PIGNNと入力を完全に揃える)
# ==========================================
class SimpleMLPClassifier(nn.Module):
    def __init__(self, in_channels=7, hidden_channels=64): # あなたのデータ(7次元)に合わせる
        super(SimpleMLPClassifier, self).__init__()
        # グラフ畳み込みを行わず、全結合層のみで判定
        self.mlp = nn.Sequential(
            nn.Linear(in_channels, hidden_channels),
            nn.BatchNorm1d(hidden_channels), # 勾配消失を防ぎ学習を安定化
            nn.ReLU(),
            nn.Linear(hidden_channels, hidden_channels),
            nn.ReLU(),
            nn.Linear(hidden_channels, 2)
        )

    def forward(self, data):
        # 重要なポイント:
        # メッセージパッシング（近接選手の相互作用理解）をバイパスし、
        # 全選手の平均的な統計量だけで予測を行う
        x = global_mean_pool(data.x, data.batch) # グラフ構造を無視した平均化
        return F.log_softmax(self.mlp(x), dim=1)

# ==========================================
# 2. MLP用のクロスバリデーション実行関数
# ==========================================
def run_mlp_cv(all_data_list, match_ids, device):
    FIXED_ALPHA = "MLP"
    # 保存先を独立させる
    model_save_dir = "/content/drive/MyDrive/GNN_Football_Analysis/Models/MLP_Baseline_take3"
    os.makedirs(model_save_dir, exist_ok=True)

    cv_final_reports = []
    print(f" MLP Baseline CV開始 (Input: 7 channels)\n")

    for test_match in match_ids:
        print(f" Round: Match {test_match} (MLP)")

        # PIGNNと全く同じルールで切り分け
        test_indices = [d for d in all_data_list if int(d.match_id.item()) == test_match]
        train_candidates = [d for d in all_data_list if int(d.match_id.item()) != test_match]

        # 論文[cite: 105]同様、訓練データのみ1:1にアンダーサンプリング
        cv_train_set = balance_dataset_by_undersampling(train_candidates)

        #cv_train_loader = DataLoader(cv_train_set, batch_size=32, shuffle=True)
        cv_train_loader = DataLoader(cv_train_set, batch_size=32, shuffle=True, drop_last=True)
        cv_test_loader = DataLoader(test_indices, batch_size=32, shuffle=False)

        # モデル初期化 (7次元)
        model = SimpleMLPClassifier(in_channels=7).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

        # 30エポック学習 (物理損失なしの標準的な学習)
        model.train()
        for epoch in range(1, 31):
            for d in cv_train_loader:
                d = d.to(device)
                optimizer.zero_grad()
                out = model(d)
                # 論文同様[cite: 115]の50/50ベースラインに近い条件でNLLLossを使用
                loss = F.nll_loss(out, d.y.view(-1))
                loss.backward()
                optimizer.step()

        # 評価 (実戦の不均衡比率のままテスト)
        model.eval()
        y_true, y_pred = [], []
        with torch.no_grad():
            for d in cv_test_loader:
                d = d.to(device)
                out = model(d)
                y_true.extend(d.y.view(-1).cpu().numpy())
                y_pred.extend(out.argmax(dim=1).cpu().numpy())

        # レポート生成
        report = classification_report(y_true, y_pred, output_dict=True, zero_division=0)
        cv_final_reports.append({
            'match': test_match,
            'recall': report['1']['recall'],
            'precision': report['1']['precision'],
            'f1': report['1']['f1-score']
        })

        # 保存 (後で可視化比較に使う)
        torch.save(model.state_dict(), os.path.join(model_save_dir, f'mlp_match_{test_match}.pth'))
        print(f" >> Result: Precision={report['1']['precision']:.4f}, F1={report['1']['f1-score']:.4f}")

    # 全体の平均を計算
    avg_precision = np.mean([r['precision'] for r in cv_final_reports])
    avg_f1 = np.mean([r['f1'] for r in cv_final_reports])

    print(f"\n MLP CV Final Results")
    print(f"Avg Precision: {avg_precision:.4f}")
    print(f"Avg F1-score:  {avg_f1:.4f}")

    return cv_final_reports

# 実行
mlp_results = run_mlp_cv(all_data_list, match_ids, device)

In [ ]:
import os
import torch
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# ==========================================
# 1. 設定：MLPモデルのロード
# ==========================================
model_save_dir = "/content/drive/MyDrive/GNN_Football_Analysis/Models/MLP_Baseline_take3"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

all_preds_mlp = []
all_labels_mlp = []
success_vxs_mlp = []

print(f" MLP Baseline 最終統合評価を開始します...")

# CVで保存した全試合のMLPモデルをロードして評価
for test_match in match_ids:
    # データの抽出
    test_indices = [d for d in all_data_list if int(d.match_id.item()) == test_match]
    test_loader = DataLoader(test_indices, batch_size=32, shuffle=False)

    # モデルの準備
    model = SimpleMLPClassifier(in_channels=7).to(device)
    model_path = os.path.join(model_save_dir, f'mlp_match_{test_match}.pth')

    if not os.path.exists(model_path):
        continue

    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    with torch.no_grad():
        for data in test_loader:
            data = data.to(device)
            out = model(data)
            pred = out.argmax(dim=1)

            # 物理的妥当性の計算
            for i in range(out.size(0)):
                mask = (data.batch == i)
                # 全ノードの平均vx（集団の推進力）
                avg_vx = torch.mean(data.vel[mask, 0]).item()

                if pred[i] == 1: # Successと予測した時のみ記録
                    success_vxs_mlp.append(avg_vx)

            all_preds_mlp.extend(pred.cpu().numpy())
            all_labels_mlp.extend(data.y.view(-1).cpu().numpy())

# ==========================================
# 2. レポート出力
# ==========================================
print("\n" + "="*60)
print("  MLP Baseline 最終クラシフィケーションレポート")
print("="*60)
print(classification_report(all_labels_mlp, all_preds_mlp, target_names=['Fail', 'Success'], zero_division=0))

print("\n" + "="*60)
print("  物理的整合性 検証 (MLP vs 物理法則)")
print("="*60)
if len(success_vxs_mlp) > 0:
    avg_vx_mlp = np.mean(success_vxs_mlp)
    pos_ratio_mlp = np.sum(np.array(success_vxs_mlp) > 0) / len(success_vxs_mlp)

    print(f"MLPが『成功』と予測したシーンの平均 vx: {avg_vx_mlp:.4f} m/s")
    print(f"右向き(攻撃方向)への推進力割合: {pos_ratio_mlp*100:.1f} %")

    # 考察用コメント
    if pos_ratio_mlp > 0.80:
        print(">> 考察: MLPは極めて高い確率で『右への速度』を成功の根拠としています。")
        print(">> これは空間構造を無視し、単純な物理量のみに依存している可能性を示唆します。")
else:
    print("Success予測なし")

# 混同行列の表示
cm = confusion_matrix(all_labels_mlp, all_preds_mlp)
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Reds', xticklabels=['Fail', 'Success'], yticklabels=['Fail', 'Success'])
plt.title('Confusion Matrix (MLP Baseline)')
plt.show()

PFI

In [ ]:
import copy
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
from sklearn.metrics import f1_score

# ==========================================
# 1. 特徴量名と重要度算出関数の定義
# ==========================================
# (ここは変更なしですが、そのまま実行できるように含めています)
feature_names = [
    'Position_X',        # Index 0
    'Position_Y',        # Index 1
    'Velocity_X',        # Index 2
    'Velocity_Y',        # Index 3
    'Distance_to_Goal',  # Index 4
    'Distance_to_Ball',  # Index 5
    'Team_Flag'          # Index 6
]

def calculate_pfi_refined(model, loader, device, feature_names, model_type="GNN"):
    model.eval()
    y_true, y_pred = [], []

    # --- Step 1: ベースラインのF1スコアを計算 ---
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            out = model(data)
            y_true.extend(data.y.view(-1).cpu().numpy())
            y_pred.extend(out.argmax(dim=1).cpu().numpy())

    baseline_f1 = f1_score(y_true, y_pred, zero_division=0)
    print(f"[{model_type}] Baseline F1: {baseline_f1:.4f}")

    importance_scores = {}

    # --- Step 2: 各特徴量を順番にシャッフルして影響を測定 ---
    for i, f_name in enumerate(feature_names):
        shuffled_f1_list = []
        for seed in range(3):
            y_true_s, y_pred_s = [], []
            with torch.no_grad():
                for data in loader:
                    data_s = copy.deepcopy(data).to(device)
                    # 特定の特徴量列をシャッフル
                    perm = torch.randperm(data_s.x.size(0))
                    data_s.x[:, i] = data_s.x[perm, i]

                    out_s = model(data_s)
                    y_true_s.extend(data_s.y.view(-1).cpu().numpy())
                    y_pred_s.extend(out_s.argmax(dim=1).cpu().numpy())

            shuffled_f1 = f1_score(y_true_s, y_pred_s, zero_division=0)
            shuffled_f1_list.append(shuffled_f1)

        importance_scores[f_name] = baseline_f1 - np.mean(shuffled_f1_list)
        print(f"  > Done: {f_name}")

    return importance_scores

# ==========================================
# 2. ロードと実行（Alpha=10.0 に修正）
# ==========================================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# モデル器の準備
pignn_model = PIGNNClassifier(hidden_channels=64).to(device)
mlp_model = SimpleMLPClassifier(in_channels=7).to(device)

# --- A. Alpha=100.0 (★提案手法・Champion Model) ---
path_10 = "/content/drive/MyDrive/GNN_Football_Analysis/Models/alpha_100_0_take3/pignn_testmatch_1.pth"
if os.path.exists(path_10):
    pignn_model.load_state_dict(torch.load(path_10, map_location=device))
    print("\n Calculating PFI for PIGNN (Alpha=100.0)...")
    pfi_10 = calculate_pfi_refined(pignn_model, cv_test_loader, device, feature_names, "Alpha100")
else:
    print(f" Path not found: {path_10}")
    pfi_10 = {f: 0 for f in feature_names} # エラー回避用ダミー

# --- B. Alpha=0.0 (物理なしGNN) ---
path_0 = "/content/drive/MyDrive/GNN_Football_Analysis/Models/alpha_0_take3/pignn_testmatch_1.pth"
if os.path.exists(path_0):
    pignn_model.load_state_dict(torch.load(path_0, map_location=device))
    print("\n Calculating PFI for PIGNN (Alpha=0)...")
    pfi_0 = calculate_pfi_refined(pignn_model, cv_test_loader, device, feature_names, "Alpha0")
else:
    pfi_0 = {f: 0 for f in feature_names}

# --- C. MLP (ベースライン) ---
path_mlp = "/content/drive/MyDrive/GNN_Football_Analysis/Models/MLP_Baseline_take3/mlp_match_1.pth"
if os.path.exists(path_mlp):
    mlp_model.load_state_dict(torch.load(path_mlp, map_location=device))
    print("\n Calculating PFI for MLP...")
    pfi_mlp = calculate_pfi_refined(mlp_model, cv_test_loader, device, feature_names, "MLP")
else:
    pfi_mlp = {f: 0 for f in feature_names}

# ==========================================
# 3. グラフ描画
# ==========================================
df_pfi = pd.DataFrame({
    'PIGNN (α=100.0)': pfi_10,  # ★ここが主役
    'PIGNN (α=0.0)': pfi_0,
    'MLP (Baseline)': pfi_mlp
})

# 色の設定（10=青, 0=水色, MLP=グレー）
colors = ['royalblue', 'skyblue', 'silver']

ax = df_pfi.plot(kind='barh', figsize=(12, 8), width=0.8, color=colors)
plt.axvline(0, color='black', linewidth=0.8)

# タイトルも修正
plt.title('Feature Importance Comparison: PIGNN(α=100.0) vs Others', fontsize=16)
plt.xlabel('Drop in F1-score (Significance)', fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.legend(loc='lower right')
plt.tight_layout()
plt.show()

In [ ]:
import copy
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
from sklearn.metrics import f1_score

# ==========================================
# 1. 特徴量名と重要度算出関数の定義
# ==========================================
# (ここは変更なしですが、そのまま実行できるように含めています)
feature_names = [
    'Position_X',        # Index 0
    'Position_Y',        # Index 1
    'Velocity_X',        # Index 2
    'Velocity_Y',        # Index 3
    'Distance_to_Goal',  # Index 4
    'Distance_to_Ball',  # Index 5
    'Team_Flag'          # Index 6
]

def calculate_pfi_refined(model, loader, device, feature_names, model_type="GNN"):
    model.eval()
    y_true, y_pred = [], []

    # --- Step 1: ベースラインのF1スコアを計算 ---
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            out = model(data)
            y_true.extend(data.y.view(-1).cpu().numpy())
            y_pred.extend(out.argmax(dim=1).cpu().numpy())

    baseline_f1 = f1_score(y_true, y_pred, zero_division=0)
    print(f"[{model_type}] Baseline F1: {baseline_f1:.4f}")

    importance_scores = {}

    # --- Step 2: 各特徴量を順番にシャッフルして影響を測定 ---
    for i, f_name in enumerate(feature_names):
        shuffled_f1_list = []
        for seed in range(3):
            y_true_s, y_pred_s = [], []
            with torch.no_grad():
                for data in loader:
                    data_s = copy.deepcopy(data).to(device)
                    # 特定の特徴量列をシャッフル
                    perm = torch.randperm(data_s.x.size(0))
                    data_s.x[:, i] = data_s.x[perm, i]

                    out_s = model(data_s)
                    y_true_s.extend(data_s.y.view(-1).cpu().numpy())
                    y_pred_s.extend(out_s.argmax(dim=1).cpu().numpy())

            shuffled_f1 = f1_score(y_true_s, y_pred_s, zero_division=0)
            shuffled_f1_list.append(shuffled_f1)

        importance_scores[f_name] = baseline_f1 - np.mean(shuffled_f1_list)
        print(f"  > Done: {f_name}")

    return importance_scores

# ==========================================
# 2. ロードと実行（Alpha=10.0 に修正）
# ==========================================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# モデル器の準備
pignn_model = PIGNNClassifier(hidden_channels=64).to(device)
mlp_model = SimpleMLPClassifier(in_channels=7).to(device)

# --- A. Alpha=100.0 (★提案手法・Champion Model) ---
path_10 = "/content/drive/MyDrive/GNN_Football_Analysis/Models/alpha_50_0_take3/pignn_testmatch_1.pth"
if os.path.exists(path_10):
    pignn_model.load_state_dict(torch.load(path_10, map_location=device))
    print("\n Calculating PFI for PIGNN (Alpha=50.0)...")
    pfi_10 = calculate_pfi_refined(pignn_model, cv_test_loader, device, feature_names, "Alpha50")
else:
    print(f" Path not found: {path_10}")
    pfi_10 = {f: 0 for f in feature_names} # エラー回避用ダミー

# --- B. Alpha=0.0 (物理なしGNN) ---
path_0 = "/content/drive/MyDrive/GNN_Football_Analysis/Models/alpha_0_take3/pignn_testmatch_1.pth"
if os.path.exists(path_0):
    pignn_model.load_state_dict(torch.load(path_0, map_location=device))
    print("\n Calculating PFI for PIGNN (Alpha=0)...")
    pfi_0 = calculate_pfi_refined(pignn_model, cv_test_loader, device, feature_names, "Alpha0")
else:
    pfi_0 = {f: 0 for f in feature_names}

# --- C. MLP (ベースライン) ---
path_mlp = "/content/drive/MyDrive/GNN_Football_Analysis/Models/MLP_Baseline_take3/mlp_match_1.pth"
if os.path.exists(path_mlp):
    mlp_model.load_state_dict(torch.load(path_mlp, map_location=device))
    print("\n Calculating PFI for MLP...")
    pfi_mlp = calculate_pfi_refined(mlp_model, cv_test_loader, device, feature_names, "MLP")
else:
    pfi_mlp = {f: 0 for f in feature_names}

# ==========================================
# 3. グラフ描画
# ==========================================
df_pfi = pd.DataFrame({
    'PIGNN (α=50.0)': pfi_10,  # ★ここが主役
    'PIGNN (α=0.0)': pfi_0,
    'MLP (Baseline)': pfi_mlp
})

# 色の設定（10=青, 0=水色, MLP=グレー）
colors = ['royalblue', 'skyblue', 'silver']

ax = df_pfi.plot(kind='barh', figsize=(12, 8), width=0.8, color=colors)
plt.axvline(0, color='black', linewidth=0.8)

# タイトルも修正
plt.title('Feature Importance Comparison: PIGNN(α=50.0) vs Others', fontsize=16)
plt.xlabel('Drop in F1-score (Significance)', fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.legend(loc='lower right')
plt.tight_layout()
plt.show()

In [ ]:
class SimpleMLPClassifier(nn.Module):
    def __init__(self, in_channels=7, hidden_channels=64): # あなたのデータ(7次元)に合わせる
        super(SimpleMLPClassifier, self).__init__()
        # グラフ畳み込みを行わず、全結合層のみで判定
        self.mlp = nn.Sequential(
            nn.Linear(in_channels, hidden_channels),
            nn.BatchNorm1d(hidden_channels), # 勾配消失を防ぎ学習を安定化
            nn.ReLU(),
            nn.Linear(hidden_channels, hidden_channels),
            nn.ReLU(),
            nn.Linear(hidden_channels, 2)
        )

    def forward(self, data):
        # 重要なポイント:
        # メッセージパッシング（近接選手の相互作用理解）をバイパスし、
        # 全選手の平均的な統計量だけで予測を行う
        x = global_mean_pool(data.x, data.batch) # グラフ構造を無視した平均化
        return F.log_softmax(self.mlp(x), dim=1)

In [ ]:
import torch
import numpy as np
from sklearn.metrics import f1_score
import copy

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ==========================================
# 構造破壊テスト関数 (Position Scramble)
# ==========================================
def test_with_position_noise(model, loader, device, model_name="Model"):
    model.eval()
    y_true, y_pred = [], []

    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            # data自体を壊さないよう、cloneを使用
            x_shuffled = data.x.clone()

            # --- 実験のキモ: 座標(index 0, 1)のみを-1~1の範囲でランダム化 ---
            num_nodes = data.x.size(0)
            noise_x = (torch.rand(num_nodes) * 2.0 - 1.0).to(device) # -1 ~ 1
            noise_y = (torch.rand(num_nodes) * 2.0 - 1.0).to(device) # -1 ~ 1

            x_shuffled[:, 0] = noise_x
            x_shuffled[:, 1] = noise_y

            # GNN用の pos 属性も同様に破壊（もしモデル内で使っている場合）
            data_copy = data.clone()
            data_copy.x = x_shuffled
            if hasattr(data_copy, 'pos'):
                data_copy.pos[:, 0] = noise_x
                data_copy.pos[:, 1] = noise_y

            # 推論
            out = model(data_copy)
            y_true.extend(data.y.view(-1).cpu().numpy())
            y_pred.extend(out.argmax(dim=1).cpu().numpy())

    f1 = f1_score(y_true, y_pred, zero_division=0)
    print(f" [{model_name}] Position-Scramble F1 Score: {f1:.4f}")
    return f1

# ==========================================
# 実行パート
# ==========================================
print(" 構造破壊 (Position Scramble) テスト開始 ⚔️")
print(">> 条件: 速度(Velocity)はそのまま。位置(Position)のみランダム化。\n")

# モデルのロード（パスは適宜調整してください）
# PIGNN (Alpha=100)
pignn_model = PIGNNClassifier(hidden_channels=64).to(device)
path_pignn = "/content/drive/MyDrive/GNN_Football_Analysis/Models/alpha_100_0_take3/pignn_testmatch_1.pth"

# MLP (Baseline)
mlp_model = SimpleMLPClassifier(in_channels=7).to(device)
path_mlp = "/content/drive/MyDrive/GNN_Football_Analysis/Models/MLP_Baseline_take3/mlp_match_1.pth"

if os.path.exists(path_pignn) and os.path.exists(path_mlp):
    pignn_model.load_state_dict(torch.load(path_pignn, map_location=device))
    mlp_model.load_state_dict(torch.load(path_mlp, map_location=device))

    # 1. PIGNNのテスト
    f1_pignn = test_with_position_noise(pignn_model, cv_test_loader, device, "PIGNN (α=100)")

    # 2. MLPのテスト
    f1_mlp = test_with_position_noise(mlp_model, cv_test_loader, device, "MLP (Baseline)")

    print("\n" + "="*40)
    print(" 結論判定")
    print("="*40)

    # 期待されるシナリオ
    # MLP: 位置が変わっても「平均速度」が変わらなければ、F1はあまり落ちないはず。
    # PIGNN: 位置が変わると「相対関係」が壊れるので、F1がガタ落ちするはず。

    if f1_mlp > f1_pignn + 0.05:
        print(f" 仮説実証成功！")
        print(f">> MLP (F1={f1_mlp:.4f}) は位置情報がデタラメでも高スコアを維持しました。")
        print("   → つまり、MLPは『配置』を見ておらず、速度計として機能しています。")
        print(f">> PIGNN (F1={f1_pignn:.4f}) はスコアが低下しました。")
        print("   → つまり、PIGNNは正しく『配置（構造）』を判断根拠にしています。")
    else:
        print(" 予想外の結果です。数値を確認してください。")

else:
    print(" モデルが見つかりません")

In [ ]:
def test_with_goal_dist_noise(model, loader, device, model_name="Model"):
    model.eval()
    y_true, y_pred = [], []

    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            x_shuffled = data.x.clone()

            # --- 実験のキモ: ゴール距離(index 4)を正規化範囲内でシャッフル ---
            num_nodes = data.x.size(0)

            # 座標等が -1~1 で正規化されている場合、それに合わせる
            # もし元のデータが 0~1 なら、noise_dist = torch.rand(num_nodes)
            noise_dist = (torch.rand(num_nodes) * 2.0 - 1.0).to(device)

            x_shuffled[:, 4] = noise_dist

            # データコピーと推論（修正なし）
            data_copy = data.clone()
            data_copy.x = x_shuffled
            out = model(data_copy)

            y_true.extend(data.y.view(-1).cpu().numpy())
            y_pred.extend(out.argmax(dim=1).cpu().numpy())

    f1 = f1_score(y_true, y_pred, zero_division=0)
    print(f" [{model_name}] Goal-Dist Scramble F1 Score: {f1:.4f}")
    return f1

print(" 空間文脈破壊 (Goal-Distance Scramble) テスト開始 ⚔️")
print(">> 条件: 速度や位置はそのまま。ゴールへの距離(dist_goal)のみランダム化。\n")

# PIGNNのテスト
f1_pignn_dist = test_with_goal_dist_noise(pignn_model, cv_test_loader, device, "PIGNN (α=100)")

# MLPのテスト
f1_mlp_dist = test_with_goal_dist_noise(mlp_model, cv_test_loader, device, "MLP (Baseline)")

print("\n" + "="*40)
print(" 解釈性に関する結論判定")
print("="*40)

if f1_pignn_dist < f1_mlp_dist:
    print(f" PIGNNの方がダメージが大きい（F1={f1_pignn_dist:.4f}）")
    print(">> PIGNNは『今どこで加速しているか（ゴールとの位置関係）』を論理的に重視しています。")
else:
    print(f" PIGNNが耐えている（F1={f1_pignn_dist:.4f}）")
    print(">> PIGNNにとってゴール距離は補助的なもので、やはり『速度($v_x$)』こそが絶対的な判断基準である可能性があります。")

アテンション係数の可視化

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import torch
import copy
import os
from torch_geometric.loader import DataLoader

# ==========================================
# 1. 可視化 & 数値抽出メイン関数
# ==========================================
def visualize_pignn_tactical_analysis(model, data_item, device, title="PIGNN Tactical Analysis"):
    model.eval()
    data_item = data_item.to(device)

    # --- 推論とアテンション抽出 ---
    with torch.no_grad():
        out, (edge_index, att_weights) = model(data_item, return_attention=True)
        prob = torch.softmax(out, dim=1)[0, 1].item()
        pred = out.argmax(dim=1).item()
        label = data_item.y.item()

    # --- 座標と速度の復元 ---
    pos = data_item.pos.cpu().numpy()
    vel = data_item.vel.cpu().numpy()
    pos_plot = np.zeros_like(pos)
    pos_plot[:, 0] = pos[:, 0] * 52.5
    pos_plot[:, 1] = pos[:, 1] * 34.0

    fig, ax = plt.subplots(figsize=(14, 10))

    # --- サッカー場の描画 (zorder=0-1) ---
    ax.set_facecolor('#2e7d32')
    ax.add_patch(patches.Rectangle((-52.5, -34), 105, 68, fill=True, color='#388e3c', zorder=0))
    ax.add_patch(patches.Rectangle((-52.5, -34), 105, 68, fill=False, color='white', lw=3, zorder=1))
    ax.plot([0, 0], [-34, 34], color='white', lw=3, zorder=1)
    ax.add_patch(patches.Circle((0, 0), 9.15, edgecolor="white", facecolor="none", lw=3, zorder=1))
    ax.add_patch(patches.Rectangle((-52.5, -20.15), 16.5, 40.3, fill=False, color='white', lw=2, zorder=1))
    ax.add_patch(patches.Rectangle((52.5-16.5, -20.15), 16.5, 40.3, fill=False, color='white', lw=2, zorder=1))

    # --- 2. アテンション係数の描画 & 数値出力 (zorder=2) ---
    att_weights = att_weights.cpu().numpy().flatten()
    edge_index = edge_index.cpu().numpy()

    if len(att_weights) > 0:
        threshold = np.percentile(att_weights, 98) # 上位2%に絞り込み
        max_att = att_weights.max()

        print(f"\n{'='*50}")
        print(f" {title} - TOP 2% ATTENTION DETAILS")
        print(f"{'='*50}")
        print(f"{'Source':<8} | {'Dest':<8} | {'Weight':<10} | {'Team Relation'}")
        print(f"{'-'*50}")

        # チームID取得 (0:攻撃, 1:守備, 2:ボール)
        team_ids = data_item.x[:, 6].cpu().numpy()

        for i in range(edge_index.shape[1]):
            if att_weights[i] > threshold:
                src, dst = edge_index[0, i], edge_index[1, i]
                weight = att_weights[i]

                # チーム関係の言語化
                rel = "Teammate" if team_ids[src] == team_ids[dst] else "Opponent"
                if team_ids[src] == 2.0: rel = "Ball -> Player"

                # コンソールに数値を表示
                print(f"Node {src:2d} -> Node {dst:2d} | {weight:.4f}     | {rel}")

                # 描画
                alpha_val = (weight - threshold) / (max_att - threshold + 1e-9)
                ax.plot([pos_plot[src, 0], pos_plot[dst, 0]],
                        [pos_plot[src, 1], pos_plot[dst, 1]],
                        color='#FFFF00', alpha=alpha_val * 0.8, lw=2.0 + alpha_val*4, zorder=2)

    # --- 3. 選手と速度ベクトルの描画 (zorder=10-20) ---
    num_nodes = pos.shape[0]
    vel_scale = 15.0 # 速度ベクトルの視認性を確保

    for i in range(num_nodes):
        if team_ids[i] == 2.0: # ボール
            color, marker, size, z = 'gold', '*', 600, 15
        elif team_ids[i] == 0.0: # 攻撃チーム
            color, marker, size, z = '#0288d1', 'o', 300, 10
        else: # 守備チーム
            color, marker, size, z = '#d32f2f', 'o', 300, 10

        # 速度ベクトル (ax.quiver で確実に描画)
        if team_ids[i] != 2.0:
            ax.quiver(pos_plot[i, 0], pos_plot[i, 1],
                      vel[i, 0], vel[i, 1],
                      color='white', alpha=0.9,
                      angles='xy', scale_units='xy', scale=1/vel_scale,
                      width=0.005, headwidth=4, headlength=5, zorder=20)

        # 選手ノード
        ax.scatter(pos_plot[i, 0], pos_plot[i, 1], c=color, marker=marker, s=size,
                   edgecolors='white', linewidth=1.5, zorder=15)

    # --- タイトルと表示設定 ---
    res_text = "SUCCESS" if pred == 1 else "FAILURE"
    match_status = "CORRECT" if label == pred else "INCORRECT"
    ax.set_title(f"{title}\nActual: {'SUCCESS' if label==1 else 'FAILURE'} | Predicted: {res_text} ({prob:.1%})\nResult: {match_status}",
                 fontsize=18, fontweight='bold', pad=20)

    ax.set_xlim(-60, 60); ax.set_ylim(-40, 40)
    plt.tight_layout()
    plt.show()

# ==========================================
# 2. 自動比較実行ループ
# ==========================================
def run_comparison_visualizer(model, data_list, device):
    success_case, failure_case = None, None
    model.eval()

    for data in data_list:
        with torch.no_grad():
            d_gpu = data.to(device)
            out, _ = model(d_gpu, return_attention=True)
            pred = out.argmax(dim=1).item()
            label = d_gpu.y.item()

            if pred == label:
                if label == 1 and success_case is None: success_case = data
                elif label == 0 and failure_case is None: failure_case = data

        if success_case and failure_case: break

    if success_case:
        visualize_pignn_tactical_analysis(model, success_case, device, title="Tactical Analysis: Successful Counter")
    if failure_case:
        visualize_pignn_tactical_analysis(model, failure_case, device, title="Tactical Analysis: Failed Counter")

# ==========================================
# 3. 実行セクション
# ==========================================
# PIGNNモデルの準備
pignn_model = PIGNNClassifier(hidden_channels=64).to(device)

pignn_path = "/content/drive/MyDrive/GNN_Football_Analysis/Models/alpha_10_0_take3/pignn_testmatch_1.pth"
if os.path.exists(pignn_path):
    state_dict = torch.load(pignn_path, map_location=device)
    pignn_model.load_state_dict(state_dict, strict=False)
    print(" PIGNN重みのロードに成功しました。")

    # 実行
    run_comparison_visualizer(pignn_model, all_data_list, device)
else:
    print(f" パスが見つかりません: {pignn_path}")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import torch
import copy
import os
from torch_geometric.loader import DataLoader

# ==========================================
# 1. 可視化 & 数値抽出メイン関数
# ==========================================
def visualize_pignn_tactical_analysis(model, data_item, device, title="PIGNN Tactical Analysis"):
    model.eval()
    data_item = data_item.to(device)

    # --- 推論とアテンション抽出 ---
    with torch.no_grad():
        out, (edge_index, att_weights) = model(data_item, return_attention=True)
        prob = torch.softmax(out, dim=1)[0, 1].item()
        pred = out.argmax(dim=1).item()
        label = data_item.y.item()

    # --- 座標と速度の復元 ---
    pos = data_item.pos.cpu().numpy()
    vel = data_item.vel.cpu().numpy()
    pos_plot = np.zeros_like(pos)
    pos_plot[:, 0] = pos[:, 0] * 52.5
    pos_plot[:, 1] = pos[:, 1] * 34.0

    fig, ax = plt.subplots(figsize=(14, 10))

    # --- サッカー場の描画 (zorder=0-1) ---
    ax.set_facecolor('#2e7d32')
    ax.add_patch(patches.Rectangle((-52.5, -34), 105, 68, fill=True, color='#388e3c', zorder=0))
    ax.add_patch(patches.Rectangle((-52.5, -34), 105, 68, fill=False, color='white', lw=3, zorder=1))
    ax.plot([0, 0], [-34, 34], color='white', lw=3, zorder=1)
    ax.add_patch(patches.Circle((0, 0), 9.15, edgecolor="white", facecolor="none", lw=3, zorder=1))
    ax.add_patch(patches.Rectangle((-52.5, -20.15), 16.5, 40.3, fill=False, color='white', lw=2, zorder=1))
    ax.add_patch(patches.Rectangle((52.5-16.5, -20.15), 16.5, 40.3, fill=False, color='white', lw=2, zorder=1))

    # --- 2. アテンション係数の描画 & 数値出力 (zorder=2) ---
    att_weights = att_weights.cpu().numpy().flatten()
    edge_index = edge_index.cpu().numpy()

    if len(att_weights) > 0:
        threshold = np.percentile(att_weights, 98) # 上位2%に絞り込み
        max_att = att_weights.max()

        print(f"\n{'='*50}")
        print(f"📊 {title} - TOP 2% ATTENTION DETAILS")
        print(f"{'='*50}")
        print(f"{'Source':<8} | {'Dest':<8} | {'Weight':<10} | {'Team Relation'}")
        print(f"{'-'*50}")

        # チームID取得 (0:攻撃, 1:守備, 2:ボール)
        team_ids = data_item.x[:, 6].cpu().numpy()

        for i in range(edge_index.shape[1]):
            if att_weights[i] > threshold:
                src, dst = edge_index[0, i], edge_index[1, i]
                weight = att_weights[i]

                # チーム関係の言語化
                rel = "Teammate" if team_ids[src] == team_ids[dst] else "Opponent"
                if team_ids[src] == 2.0: rel = "Ball -> Player"

                # コンソールに数値を表示
                print(f"Node {src:2d} -> Node {dst:2d} | {weight:.4f}     | {rel}")

                # 描画
                alpha_val = (weight - threshold) / (max_att - threshold + 1e-9)
                ax.plot([pos_plot[src, 0], pos_plot[dst, 0]],
                        [pos_plot[src, 1], pos_plot[dst, 1]],
                        color='#FFFF00', alpha=alpha_val * 0.8, lw=2.0 + alpha_val*4, zorder=2)

    # --- 3. 選手と速度ベクトルの描画 (zorder=10-20) ---
    num_nodes = pos.shape[0]
    vel_scale = 15.0 # 速度ベクトルの視認性を確保

    for i in range(num_nodes):
        if team_ids[i] == 2.0: # ボール
            color, marker, size, z = 'gold', '*', 600, 15
        elif team_ids[i] == 0.0: # 攻撃チーム
            color, marker, size, z = '#0288d1', 'o', 300, 10
        else: # 守備チーム
            color, marker, size, z = '#d32f2f', 'o', 300, 10

        # 速度ベクトル (ax.quiver で確実に描画)
        if team_ids[i] != 2.0:
            ax.quiver(pos_plot[i, 0], pos_plot[i, 1],
                      vel[i, 0], vel[i, 1],
                      color='white', alpha=0.9,
                      angles='xy', scale_units='xy', scale=1/vel_scale,
                      width=0.005, headwidth=4, headlength=5, zorder=20)

        # 選手ノード
        ax.scatter(pos_plot[i, 0], pos_plot[i, 1], c=color, marker=marker, s=size,
                   edgecolors='white', linewidth=1.5, zorder=15)

    # --- タイトルと表示設定 ---
    res_text = "SUCCESS" if pred == 1 else "FAILURE"
    match_status = "CORRECT" if label == pred else "INCORRECT"
    ax.set_title(f"{title}\nActual: {'SUCCESS' if label==1 else 'FAILURE'} | Predicted: {res_text} ({prob:.1%})\nResult: {match_status}",
                 fontsize=18, fontweight='bold', pad=20)

    ax.set_xlim(-60, 60); ax.set_ylim(-40, 40)
    plt.tight_layout()
    plt.show()

# ==========================================
# 2. 自動比較実行ループ
# ==========================================
def run_comparison_visualizer(model, data_list, device):
    success_case, failure_case = None, None
    model.eval()

    for data in data_list:
        with torch.no_grad():
            d_gpu = data.to(device)
            out, _ = model(d_gpu, return_attention=True)
            pred = out.argmax(dim=1).item()
            label = d_gpu.y.item()

            if pred == label:
                if label == 1 and success_case is None: success_case = data
                elif label == 0 and failure_case is None: failure_case = data

        if success_case and failure_case: break

    if success_case:
        visualize_pignn_tactical_analysis(model, success_case, device, title="Tactical Analysis: Successful Counter")
    if failure_case:
        visualize_pignn_tactical_analysis(model, failure_case, device, title="Tactical Analysis: Failed Counter")

# ==========================================
# 3. 実行セクション
# ==========================================
# PIGNNモデルの準備
pignn_model = PIGNNClassifier(hidden_channels=64).to(device)

pignn_path = "/content/drive/MyDrive/GNN_Football_Analysis/Models/alpha_100_0_take3/pignn_testmatch_3.pth"
if os.path.exists(pignn_path):
    state_dict = torch.load(pignn_path, map_location=device)
    pignn_model.load_state_dict(state_dict, strict=False)
    print(" PIGNN重みのロードに成功しました。")

    # 実行
    run_comparison_visualizer(pignn_model, all_data_list, device)
else:
    print(f" パスが見つかりません: {pignn_path}")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import torch
import copy
import os
from torch_geometric.loader import DataLoader

# ==========================================
# 1. 可視化 & 数値抽出メイン関数
# ==========================================
def visualize_pignn_tactical_analysis(model, data_item, device, title="PIGNN Tactical Analysis"):
    model.eval()
    data_item = data_item.to(device)

    # --- 推論とアテンション抽出 ---
    with torch.no_grad():
        out, (edge_index, att_weights) = model(data_item, return_attention=True)
        prob = torch.softmax(out, dim=1)[0, 1].item()
        pred = out.argmax(dim=1).item()
        label = data_item.y.item()

    # --- 座標と速度の復元 ---
    pos = data_item.pos.cpu().numpy()
    vel = data_item.vel.cpu().numpy()
    pos_plot = np.zeros_like(pos)
    pos_plot[:, 0] = pos[:, 0] * 52.5
    pos_plot[:, 1] = pos[:, 1] * 34.0

    fig, ax = plt.subplots(figsize=(14, 10))

    # --- サッカー場の描画 (zorder=0-1) ---
    ax.set_facecolor('#2e7d32')
    ax.add_patch(patches.Rectangle((-52.5, -34), 105, 68, fill=True, color='#388e3c', zorder=0))
    ax.add_patch(patches.Rectangle((-52.5, -34), 105, 68, fill=False, color='white', lw=3, zorder=1))
    ax.plot([0, 0], [-34, 34], color='white', lw=3, zorder=1)
    ax.add_patch(patches.Circle((0, 0), 9.15, edgecolor="white", facecolor="none", lw=3, zorder=1))
    ax.add_patch(patches.Rectangle((-52.5, -20.15), 16.5, 40.3, fill=False, color='white', lw=2, zorder=1))
    ax.add_patch(patches.Rectangle((52.5-16.5, -20.15), 16.5, 40.3, fill=False, color='white', lw=2, zorder=1))

    # --- 2. アテンション係数の描画 & 数値出力 (zorder=2) ---
    att_weights = att_weights.cpu().numpy().flatten()
    edge_index = edge_index.cpu().numpy()

    if len(att_weights) > 0:
        threshold = np.percentile(att_weights, 98) # 上位2%に絞り込み
        max_att = att_weights.max()

        print(f"\n{'='*50}")
        print(f"📊 {title} - TOP 2% ATTENTION DETAILS")
        print(f"{'='*50}")
        print(f"{'Source':<8} | {'Dest':<8} | {'Weight':<10} | {'Team Relation'}")
        print(f"{'-'*50}")

        # チームID取得 (0:攻撃, 1:守備, 2:ボール)
        team_ids = data_item.x[:, 6].cpu().numpy()

        for i in range(edge_index.shape[1]):
            if att_weights[i] > threshold:
                src, dst = edge_index[0, i], edge_index[1, i]
                weight = att_weights[i]

                # チーム関係の言語化
                rel = "Teammate" if team_ids[src] == team_ids[dst] else "Opponent"
                if team_ids[src] == 2.0: rel = "Ball -> Player"

                # コンソールに数値を表示
                print(f"Node {src:2d} -> Node {dst:2d} | {weight:.4f}     | {rel}")

                # 描画
                alpha_val = (weight - threshold) / (max_att - threshold + 1e-9)
                ax.plot([pos_plot[src, 0], pos_plot[dst, 0]],
                        [pos_plot[src, 1], pos_plot[dst, 1]],
                        color='#FFFF00', alpha=alpha_val * 0.8, lw=2.0 + alpha_val*4, zorder=2)

    # --- 3. 選手と速度ベクトルの描画 (zorder=10-20) ---
    num_nodes = pos.shape[0]
    vel_scale = 15.0 # 速度ベクトルの視認性を確保

    for i in range(num_nodes):
        if team_ids[i] == 2.0: # ボール
            color, marker, size, z = 'gold', '*', 600, 15
        elif team_ids[i] == 0.0: # 攻撃チーム
            color, marker, size, z = '#0288d1', 'o', 300, 10
        else: # 守備チーム
            color, marker, size, z = '#d32f2f', 'o', 300, 10

        # 速度ベクトル (ax.quiver で確実に描画)
        if team_ids[i] != 2.0:
            ax.quiver(pos_plot[i, 0], pos_plot[i, 1],
                      vel[i, 0], vel[i, 1],
                      color='white', alpha=0.9,
                      angles='xy', scale_units='xy', scale=1/vel_scale,
                      width=0.005, headwidth=4, headlength=5, zorder=20)

        # 選手ノード
        ax.scatter(pos_plot[i, 0], pos_plot[i, 1], c=color, marker=marker, s=size,
                   edgecolors='white', linewidth=1.5, zorder=15)

    # --- タイトルと表示設定 ---
    res_text = "SUCCESS" if pred == 1 else "FAILURE"
    match_status = "CORRECT" if label == pred else "INCORRECT"
    ax.set_title(f"{title}\nActual: {'SUCCESS' if label==1 else 'FAILURE'} | Predicted: {res_text} ({prob:.1%})\nResult: {match_status}",
                 fontsize=18, fontweight='bold', pad=20)

    ax.set_xlim(-60, 60); ax.set_ylim(-40, 40)
    plt.tight_layout()
    plt.show()

# ==========================================
# 2. 自動比較実行ループ
# ==========================================
def run_comparison_visualizer(model, data_list, device):
    success_case, failure_case = None, None
    model.eval()

    for data in data_list:
        with torch.no_grad():
            d_gpu = data.to(device)
            out, _ = model(d_gpu, return_attention=True)
            pred = out.argmax(dim=1).item()
            label = d_gpu.y.item()

            if pred == label:
                if label == 1 and success_case is None: success_case = data
                elif label == 0 and failure_case is None: failure_case = data

        if success_case and failure_case: break

    if success_case:
        visualize_pignn_tactical_analysis(model, success_case, device, title="Tactical Analysis: Successful Counter")
    if failure_case:
        visualize_pignn_tactical_analysis(model, failure_case, device, title="Tactical Analysis: Failed Counter")

# ==========================================
# 3. 実行セクション
# ==========================================
# PIGNNモデルの準備
pignn_model = PIGNNClassifier(hidden_channels=64).to(device)

pignn_path = "/content/drive/MyDrive/GNN_Football_Analysis/Models/alpha_100_0_take3/pignn_testmatch_2.pth"
if os.path.exists(pignn_path):
    state_dict = torch.load(pignn_path, map_location=device)
    pignn_model.load_state_dict(state_dict, strict=False)
    print(" PIGNN重みのロードに成功しました。")

    # 実行
    run_comparison_visualizer(pignn_model, all_data_list, device)
else:
    print(f" パスが見つかりません: {pignn_path}")

In [ ]:
#Explainerのために合わせる

class PIGNNClassifier(nn.Module):
    def __init__(self, hidden_channels=64):
        super(PIGNNClassifier, self).__init__()
        self.conv1 = PIGNNLayer(7, hidden_channels, tau=0.2)
        self.conv2 = PIGNNLayer(hidden_channels, hidden_channels, tau=0.2)
        self.lin = nn.Linear(hidden_channels, 2)

    #  GNNExplainerがx, edge_index, batchなどを個別に渡せるように修正
    def forward(self, x, edge_index, batch=None, pos=None, vel=None, return_attention=False, **kwargs):
        # batchがNoneの場合は単一グラフとして扱う
        if batch is None:
            batch = torch.zeros(x.size(0), dtype=torch.long, device=x.device)

        if return_attention:
            x, (edge_idx_out, att_weights) = self.conv1(x, edge_index, pos, vel, return_attention=True)
            x = F.elu(x)
            x = self.conv2(x, edge_index, pos, vel)
        else:
            x = F.elu(self.conv1(x, edge_index, pos, vel))
            x = self.conv2(x, edge_index, pos, vel)

        x_pool = global_mean_pool(x, batch)
        logits = F.log_softmax(self.lin(x_pool), dim=1)

        if return_attention:
            return logits, (edge_idx_out, att_weights)
        return logits

In [ ]:
from torch_geometric.explain import Explainer, GNNExplainer

# モデルのインスタンス化と重みのロード（パスはあなたの環境に合わせて）
pignn_model = PIGNNClassifier(hidden_channels=64).to(device)
pignn_model.load_state_dict(torch.load(pignn_path))

# Explainerの設定（ここは以前と同じ）
explainer = Explainer(
    model=pignn_model,
    algorithm=GNNExplainer(epochs=200),
    explanation_type='model',
    node_mask_type='attributes',
    edge_mask_type='object',
    model_config=dict(
        mode='multiclass_classification',
        task_level='graph',
        return_type='log_probs',
    ),
)

# 実行
data_single = all_data_list[0].to(device)
explanation = explainer(
    x=data_single.x,
    edge_index=data_single.edge_index,
    batch=torch.zeros(data_single.x.size(0), dtype=torch.long, device=device),
    pos=data_single.pos,
    vel=data_single.vel
)

In [ ]:
import matplotlib.pyplot as plt

# 特徴量ラベル（あなたのデータ形式に合わせて調整してください）
# 例: 0:x, 1:y, 2:vx, 3:vy, 4:dist_goal, 5:dist_ball, 6:team_id ...
labels = [
    'Position_X',        # Index 0
    'Position_Y',        # Index 1
    'Velocity_X',        # Index 2
    'Velocity_Y',        # Index 3
    'Distance_to_Goal',  # Index 4
    'Distance_to_Ball',  # Index 5
    'Team_Flag'          # Index 6
]

# 特徴量ごとの重要度（node_mask）を全ノードで集計
feat_importance = explanation.node_mask.abs().sum(dim=0).cpu().numpy()

plt.figure(figsize=(10, 6))
plt.bar(labels[:len(feat_importance)], feat_importance)
plt.title("Feature Importance (GNNExplainer)")
plt.ylabel("Importance Score")
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt

def explain_and_visualize_comparison(explainer, data_list, device):
    # --- 1. 成功シーン(Probが高い)と失敗シーン(Probが低い)を探索 ---
    target_success_idx = -1
    target_failure_idx = -1

    print(" Searching for comparison cases...")
    pignn_model.eval()
    with torch.no_grad():
        for i, data in enumerate(data_list):
            data = data.to(device)
            if not hasattr(data, 'batch') or data.batch is None:
                data.batch = torch.zeros(data.x.size(0), dtype=torch.long, device=device)

            out = pignn_model(data)
            prob = torch.softmax(out, dim=1)[0, 1].item()

            if prob > 0.9 and target_success_idx == -1:
                target_success_idx = i
            if prob < 0.1 and target_failure_idx == -1:
                target_failure_idx = i

            if target_success_idx != -1 and target_failure_idx != -1:
                break

    # 表示名を英語に変更して文字化けを回避
    indices = {"AI Prediction: SUCCESS": target_success_idx, "AI Prediction: FAILURE": target_failure_idx}

    # --- 2. 順に可視化 ---
    for title_prefix, idx in indices.items():
        if idx == -1:
            print(f" {title_prefix} matching scene not found.")
            continue

        data_single = data_list[idx].to(device)
        print(f"\n🎬 Analyzing {title_prefix} (Index: {idx})...")

        explanation = explainer(
            x=data_single.x,
            edge_index=data_single.edge_index,
            batch=torch.zeros(data_single.x.size(0), dtype=torch.long, device=device),
            pos=data_single.pos,
            vel=data_single.vel
        )

        # --- A. 棒グラフ (特徴量重要度) ---
        feat_importance = explanation.node_mask.abs().sum(dim=0).cpu().numpy()
        plt.figure(figsize=(8, 4))
        # 成功ならSkyblue, 失敗ならSalmonで色分け
        bar_color = 'skyblue' if "SUCCESS" in title_prefix else 'salmon'
        plt.bar(labels[:len(feat_importance)], feat_importance, color=bar_color)
        plt.title(f"Feature Importance: {title_prefix}")
        plt.ylabel("Importance Score")
        plt.xticks(rotation=30)
        plt.grid(axis='y', linestyle='--', alpha=0.5) # グリッドを追加して見やすく
        plt.tight_layout() # ラベルの切れを防止
        plt.show()

        # --- B. ピッチ図 (戦術可視化) ---
        visualize_explanation_on_pitch_fixed(
            explanation, data_single,
            title=f"{title_prefix}\n(GT: {'Success' if data_single.y.item()==1 else 'Failure'})"
        )

# 実行
explain_and_visualize_comparison(explainer, all_data_list, device)

In [ ]:
# 重要度の高いエッジのインデックスを抽出
edge_mask = explanation.edge_mask.cpu().numpy()
edge_index = data_single.edge_index.cpu().numpy()

# 上位5%のエッジを抽出
threshold = np.percentile(edge_mask, 95)

# この上位エッジを、以前作成した「ピッチ可視化コード」の
# 黄色い線の代わりに描画すれば、「AIが選んだ真の戦術」が浮かび上がります。

In [ ]:
def visualize_explanation_on_pitch_fixed(explanation, data, title="GNNExplainer Tactical Insight"):
    # --- 1. 座標の変換ロジックを修正 ---
    pos = data.pos.cpu().numpy()

    # 正規化範囲が -1 ~ 1 の場合：
    # そのままピッチサイズ（半分）を掛けるだけで、中央(0,0)を基準に配置されます
    pos_plot_x = pos[:, 0] * 52.5  # -1*52.5 = -52.5 (左端), 1*52.5 = 52.5 (右端)
    pos_plot_y = pos[:, 1] * 34.0  # -1*34.0 = -34.0 (下端), 1*34.0 = 34.0 (上端)

    edge_index = data.edge_index.cpu().numpy()
    edge_mask = explanation.edge_mask.cpu().numpy()

    # チームID取得 (xの7番目: index 6)
    # 0.0:攻撃, 1.0:守備, 2.0:ボール
    team_ids = data.x[:, 6].cpu().numpy()

    # --- 2. ピッチ描画 (背景) ---
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.set_facecolor('#2e7d32')
    ax.add_patch(patches.Rectangle((-52.5, -34), 105, 68, fill=True, color='#388e3c', zorder=0))
    ax.add_patch(patches.Rectangle((-52.5, -34), 105, 68, fill=False, color='white', lw=2, zorder=1))
    ax.plot([0, 0], [-34, 34], color='white', lw=2, zorder=1)
    ax.add_patch(patches.Circle((0, 0), 9.15, edgecolor="white", facecolor="none", lw=2, zorder=1))

    # --- 3. 重要エッジの描画 ---
    threshold = np.percentile(edge_mask, 98)
    max_val = edge_mask.max()

    for i in range(len(edge_mask)):
        if edge_mask[i] >= threshold:
            src, dst = edge_index[0, i], edge_index[1, i]
            val_norm = (edge_mask[i] - threshold) / (max_val - threshold + 1e-9)
            ax.plot([pos_plot_x[src], pos_plot_x[dst]],
                    [pos_plot_y[src], pos_plot_y[dst]],
                    color='yellow', alpha=0.4 + 0.6*val_norm,
                    lw=2 + 6*val_norm, zorder=2)

    # --- 4. 選手・ボールの描画 ---
    for i in range(len(pos)):
        # チームラベルに基づいた色分け
        if team_ids[i] == 2.0: # ボール
            color, marker, size = 'gold', '*', 500
        elif team_ids[i] == 0.0: # 攻撃
            color, marker, size = '#0288d1', 'o', 250
        else: # 守備
            color, marker, size = '#d32f2f', 'o', 250

        ax.scatter(pos_plot_x[i], pos_plot_y[i], c=color, marker=marker,
                   s=size, edgecolors='white', linewidth=1.5, zorder=3)

    ax.set_title(title, fontsize=15, pad=20)
    ax.set_xlim(-60, 60); ax.set_ylim(-40, 40)
    plt.show()

# 実行
visualize_explanation_on_pitch_fixed(explanation, data_single)

In [ ]:
print(f"モデルの予測ラベル: {explanation.prediction.argmax().item()} (0:失敗, 1:成功)")
print(f"実際の正解ラベル: {data_single.y.item()} (0:失敗, 1:成功)")

In [ ]:
def find_and_visualize_tp_final(model, dataset, num_samples=1):
    # モデルのデバイスを取得
    device = next(model.parameters()).device
    model.eval()
    tp_indices = []

    print(f" 成功的中(True Positive)シーンを探索中... (Device: {device})")
    for i, data in enumerate(dataset):
        # データをモデルと同じデバイスに転送
        x = data.x.to(device).to(torch.float)
        edge_index = data.edge_index.to(device).to(torch.long)
        pos = data.pos.to(device).to(torch.float)
        vel = data.vel.to(device).to(torch.float)

        with torch.no_grad():
            # forwardの引数名はあなたのモデル定義に合わせて適宜調整してください
            # ここでは (x, edge_index, pos, vel) を想定
            out = model(x, edge_index, pos=pos, vel=vel)

            pred = out.argmax(dim=1).item()
            actual = data.y.item()

            if pred == 1 and actual == 1:
                prob = torch.exp(out[0][1]).item()
                tp_indices.append((i, prob))

    tp_indices = sorted(tp_indices, key=lambda x: x[1], reverse=True)

    if not tp_indices:
        print("❌ 成功的中シーンが見つかりませんでした。")
        return

    print(f" {len(tp_indices)} 件の成功的中シーンが見つかりました。上位を可視化します。")

    from torch_geometric.explain import Explainer, GNNExplainer

    # Explainer用のラッパー（デバイス対応）
    class ExplainerWrapper(torch.nn.Module):
        def __init__(self, model):
            super().__init__()
            self.model = model

        def forward(self, x, edge_index, **kwargs):
            # Explainerから渡される座標や速度もデバイスを合わせる
            pos = kwargs.get('pos')
            vel = kwargs.get('vel')
            return self.model(x, edge_index, pos=pos, vel=vel)

    wrapper = ExplainerWrapper(model)

    explainer = Explainer(
        model=wrapper,
        algorithm=GNNExplainer(epochs=200),
        explanation_type='model',
        node_mask_type='attributes',
        edge_mask_type='object',
        model_config=dict(
            mode='multiclass_classification',
            task_level='graph',
            return_type='log_probs',
        ),
    )

    for i in range(min(num_samples, len(tp_indices))):
        idx, prob = tp_indices[i]
        # 元データをGPUに送ってからExplainerに渡す
        data_single = dataset[idx].to(device)

        explanation = explainer(
            data_single.x,
            data_single.edge_index,
            pos=data_single.pos,
            vel=data_single.vel
        )

        title = f"TRUE POSITIVE (AI Correct!)\nActual: Success, Pred: Success (Prob: {prob:.2f})"
        visualize_explanation_on_pitch_fixed(explanation, data_single, title=title)
        print(f" Index {idx} の可視化が完了しました。")

# 実行
find_and_visualize_tp_final(pignn_model, all_data_list, num_samples=3)

In [ ]:
def visualize_tp_by_rank(model, dataset, rank=0):
    """
    rank: 0なら最も確率が高いシーン、1なら2番目...を表示
    """
    device = next(model.parameters()).device
    model.eval()
    tp_indices = []

    # 1. まず全ての成功的中(TP)シーンを抽出
    print(f" 成功的中(True Positive)シーンをランキング化中...")
    for i, data in enumerate(dataset):
        x = data.x.to(device).to(torch.float)
        edge_index = data.edge_index.to(device).to(torch.long)
        pos = data.pos.to(device).to(torch.float)
        vel = data.vel.to(device).to(torch.float)

        with torch.no_grad():
            out = model(x, edge_index, pos=pos, vel=vel)
            pred = out.argmax(dim=1).item()
            actual = data.y.item()

            if pred == 1 and actual == 1:
                # 成功(index 1)の確率を取得
                prob = torch.exp(out[0][1]).item()
                tp_indices.append((i, prob))

    # 2. 確率が高い順にソート
    tp_indices = sorted(tp_indices, key=lambda x: x[1], reverse=True)

    if not tp_indices:
        print("❌ 成功的中シーンが見つかりませんでした。")
        return

    if rank >= len(tp_indices):
        print(f"❌ 指定された順位(rank={rank})は範囲外です。全{len(tp_indices)}件です。")
        return

    # 3. 指定された順位のシーンを1枚だけ詳しく解析
    idx, prob = tp_indices[rank]
    print(f" 自信度ランキング第 {rank+1} 位 (Index: {idx}, Prob: {prob:.4f}) を可視化します。")

    from torch_geometric.explain import Explainer, GNNExplainer

    class ExplainerWrapper(torch.nn.Module):
        def __init__(self, model):
            super().__init__()
            self.model = model
        def forward(self, x, edge_index, **kwargs):
            return self.model(x, edge_index, pos=kwargs.get('pos'), vel=kwargs.get('vel'))

    wrapper = ExplainerWrapper(model)
    explainer = Explainer(
        model=wrapper,
        algorithm=GNNExplainer(epochs=200), # 200あれば十分です
        explanation_type='model',
        node_mask_type='attributes',
        edge_mask_type='object',
        model_config=dict(mode='multiclass_classification', task_level='graph', return_type='log_probs'),
    )

    data_single = dataset[idx].to(device)
    explanation = explainer(data_single.x, data_single.edge_index, pos=data_single.pos, vel=data_single.vel)

    title = f"TRUE POSITIVE - Rank: {rank+1}\nActual: Success, Pred: Success (Prob: {prob:.3f})\nData Index: {idx}"
    visualize_explanation_on_pitch_fixed(explanation, data_single, title=title)

# --- 実行エリア ---
# rank=0 で最強の成功シーン、rank=1, 2... と変えて実行してください
visualize_tp_by_rank(pignn_model, all_data_list, rank=30)

In [ ]:
def find_and_visualize_by_case(model, dataset, case_type='TP', num_samples=1):
    """
    case_type:
        'TP' (True Positive) : 実際成功 / 予測成功 (的中)
        'TN' (True Negative) : 実際失敗 / 予測失敗 (的中) -> 失敗シーンの的中
        'FP' (False Positive): 実際失敗 / 予測成功 (空振り) -> 失敗シーンの誤認
        'FN' (False Negative): 実際成功 / 予測失敗 (見逃し)
    """
    device = next(model.parameters()).device
    model.eval()
    indices = []

    print(f"🔎 {case_type} シーンを探索中... (Device: {device})")

    for i, data in enumerate(dataset):
        x = data.x.to(device).to(torch.float)
        edge_index = data.edge_index.to(device).to(torch.long)
        pos = data.pos.to(device).to(torch.float)
        vel = data.vel.to(device).to(torch.float)

        with torch.no_grad():
            out = model(x, edge_index, pos=pos, vel=vel)
            pred = out.argmax(dim=1).item()
            actual = data.y.item()
            prob = torch.exp(out[0][pred]).item() # 予測したクラスの確信度

            # 条件判定
            is_match = False
            if case_type == 'TP' and pred == 1 and actual == 1: is_match = True
            elif case_type == 'TN' and pred == 0 and actual == 0: is_match = True
            elif case_type == 'FP' and pred == 1 and actual == 0: is_match = True
            elif case_type == 'FN' and pred == 0 and actual == 1: is_match = True

            if is_match:
                indices.append((i, prob))

    # 確信度（確率）が高い順にソート
    indices = sorted(indices, key=lambda x: x[1], reverse=True)

    if not indices:
        print(f"❌ {case_type} シーンが見つかりませんでした。")
        return

    print(f" {len(indices)} 件の {case_type} シーンが見つかりました。上位を可視化します。")

    # Explainerの設定（デバイス対応）
    from torch_geometric.explain import Explainer, GNNExplainer
    class ExplainerWrapper(torch.nn.Module):
        def __init__(self, model):
            super().__init__()
            self.model = model
        def forward(self, x, edge_index, **kwargs):
            return self.model(x, edge_index, pos=kwargs.get('pos'), vel=kwargs.get('vel'))

    explainer = Explainer(
        model=ExplainerWrapper(model),
        algorithm=GNNExplainer(epochs=200),
        explanation_type='model',
        node_mask_type='attributes',
        edge_mask_type='object',
        model_config=dict(
            mode='multiclass_classification',
            task_level='graph',
            return_type='log_probs',
        ),
    )

    for i in range(min(num_samples, len(indices))):
        idx, prob = indices[i]
        data_single = dataset[idx].to(device)

        explanation = explainer(
            data_single.x,
            data_single.edge_index,
            pos=data_single.pos,
            vel=data_single.vel
        )

        # ケースに応じたタイトル作成
        status_map = {
            'TP': "TRUE POSITIVE (AI Correct! - Success)",
            'TN': "TRUE NEGATIVE (AI Correct! - Failure)",
            'FP': "FALSE POSITIVE (AI Mistake! - Predicted Success but Failed)",
            'FN': "FALSE NEGATIVE (AI Mistake! - Predicted Failure but Successed)"
        }
        actual_label = "Success" if data_single.y.item() == 1 else "Failure"
        pred_label = "Success" if case_type in ['TP', 'FP'] else "Failure"

        title = f"{status_map[case_type]}\nActual: {actual_label}, Pred: {pred_label} (Prob: {prob:.2f})"

        visualize_explanation_on_pitch_fixed(explanation, data_single, title=title)
        print(f" Index {idx} ({case_type}) の可視化が完了しました。")

# --- 使い方 ---

# 1. 失敗を失敗と正しく判定したシーン（なぜ失敗するとAIが判断したかの根拠）
find_and_visualize_by_case(pignn_model, all_data_list, case_type='TN', num_samples=3)

# 2. 失敗したのに成功と誤認したシーン（AIがどこに騙されたかの根拠）
find_and_visualize_by_case(pignn_model, all_data_list, case_type='FP', num_samples=3)

In [ ]:
import torch
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from torch_geometric.loader import DataLoader

# ==========================================
# 1. 描画関数の再定義（引数エラー修正版）
# ==========================================
def visualize_pignn_tactical_analysis(model, data_item, device, title="Analysis"):
    model.eval()
    data_item = data_item.to(device)

    # --- 推論とアテンション抽出（完全指定版） ---
    with torch.no_grad():
        out, (edge_index, att_weights) = model(
            data=data_item,
            x=data_item.x,
            edge_index=data_item.edge_index,
            batch=data_item.batch,
            pos=data_item.pos,
            vel=data_item.vel,
            return_attention=True
        )
        prob = torch.softmax(out, dim=1)[0, 1].item()
        pred = out.argmax(dim=1).item()
        label = data_item.y.item()

    # --- 座標と速度の復元 ---
    pos = data_item.pos.cpu().numpy()
    vel = data_item.vel.cpu().numpy()
    # 座標スケール調整（ピッチサイズに合わせる）
    pos_plot = np.zeros_like(pos)
    pos_plot[:, 0] = pos[:, 0] * 52.5
    pos_plot[:, 1] = pos[:, 1] * 34.0

    fig, ax = plt.subplots(figsize=(12, 8))

    # サッカーピッチの描画
    ax.set_facecolor('#2e7d32')
    ax.add_patch(patches.Rectangle((-52.5, -34), 105, 68, fill=True, color='#388e3c', zorder=0))
    ax.add_patch(patches.Rectangle((-52.5, -34), 105, 68, fill=False, color='white', lw=2, zorder=1))
    ax.plot([0, 0], [-34, 34], color='white', lw=2, zorder=1)

    # アテンション係数の描画
    att_weights = att_weights.cpu().numpy().flatten()
    edge_index = edge_index.cpu().numpy()
    if len(att_weights) > 0:
        threshold = np.percentile(att_weights, 98) # 上位2%を可視化
        for i in range(edge_index.shape[1]):
            if att_weights[i] > threshold:
                src, dst = edge_index[0, i], edge_index[1, i]
                alpha_val = min((att_weights[i] / att_weights.max()), 1.0)
                ax.plot([pos_plot[src, 0], pos_plot[dst, 0]],
                        [pos_plot[src, 1], pos_plot[dst, 1]],
                        color='#FFFF00', alpha=alpha_val, lw=2, zorder=2)

    # 選手ノードの描画
    team_ids = data_item.x[:, 6].cpu().numpy()
    for i in range(len(pos)):
        if team_ids[i] == 2.0: # ボール
            color, size, marker = 'gold', 400, '*'
        elif team_ids[i] == 0.0: # 攻撃
            color, size, marker = '#0288d1', 200, 'o'
        else: # 守備
            color, size, marker = '#d32f2f', 200, 'o'

        ax.scatter(pos_plot[i, 0], pos_plot[i, 1], c=color, s=size, marker=marker,
                   edgecolors='white', linewidth=1, zorder=10)

    ax.set_title(f"{title}\nSuccess Prob: {prob:.1%}", fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

# ==========================================
# 2. データの「完全CPU化」と再構成
# ==========================================
print(" データのデバイス整合性を修正中...")
if 'all_data_list' in locals():
    all_data_list_cpu = [d.to('cpu') for d in all_data_list]
    cv_test_loader = DataLoader(all_data_list_cpu, batch_size=1, shuffle=False)
    print(" 全データをCPUに配置し、cv_test_loaderを再構成しました。")
else:
    print(" all_data_listが見つかりません。")

# ==========================================
# 3. 戦略的比較関数の定義（マイルドな閾値設定）
# ==========================================
def run_strategic_comparison(model, loader, device):
    model.eval()
    all_results = []

    print(" 比較シーンを探索中... (Device: {})".format(device))

    with torch.no_grad():
        for data in loader:
            data = data.to('cpu')
            data = preprocess_batch(data, device)

            if not hasattr(data, 'batch') or data.batch is None:
                data.batch = torch.zeros(data.x.size(0), dtype=torch.long, device=device)

            # モデル推論
            out, (edge_idx, att) = model(
                data=data, x=data.x, edge_index=data.edge_index,
                batch=data.batch, pos=data.pos, vel=data.vel, return_attention=True
            )
            prob = torch.softmax(out, dim=1)[0, 1].item()

            atk_mask = (data.x[:, 6] == 0.0)
            avg_vx = data.vel[atk_mask, 0].mean().item() if atk_mask.any() else 0.0

            all_results.append({
                'data': data.cpu(),
                'avg_vx': avg_vx,
                'prob': prob,
                'label': data.y.item()
            })

    if not all_results:
        print(" データが見つかりません。")
        return

    # 条件A: 速度はあるが、AIが失敗と予測（戦術的失敗）
    # 閾値を0.2以上 / 0.5以下にマイルド化
    candidates_a = [res for res in all_results if res['avg_vx'] > 0.10 and res['prob'] < 0.5]
    case_a = sorted(candidates_a, key=lambda x: x['prob'])[0] if candidates_a else None

    # 条件B: 速度は平凡だが、AIが成功と予測（戦術的成功）
    # 閾値を0.2以下 / 0.6以上にマイルド化
    candidates_b = [res for res in all_results if res['avg_vx'] < 0.2 and res['prob'] > 0.6]
    case_b = sorted(candidates_b, key=lambda x: x['prob'], reverse=True)[0] if candidates_b else None

    # 描画
    if case_a:
        print(f" 条件A発見: Avg_Vx={case_a['avg_vx']:.3f}, Prob={case_a['prob']:.3f}")
        visualize_pignn_tactical_analysis(model, case_a['data'], device, title="Condition A: High Physics but Tactical Failure")
    else:
        print(" 条件Aが見つかりませんでした。")

    if case_b:
        print(f" 条件B発見: Avg_Vx={case_b['avg_vx']:.3f}, Prob={case_b['prob']:.3f}")
        visualize_pignn_tactical_analysis(model, case_b['data'], device, title="Condition B: Normal Physics but Tactical Success")
    else:
        print(" 条件Bが見つかりませんでした。")

# ==========================================
# 4. 実行セクション
# ==========================================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pignn_model = PIGNNClassifier(hidden_channels=64).to(device)
pignn_path = "/content/drive/MyDrive/GNN_Football_Analysis/Models/alpha_100_0_take3/pignn_testmatch_1.pth"

if os.path.exists(pignn_path):
    pignn_model.load_state_dict(torch.load(pignn_path, map_location=device), strict=False)
    print(" PIGNN重みロード完了。解析を開始します。")
    run_strategic_comparison(pignn_model, cv_test_loader, device)
else:
    print(f" ファイル未検出: {pignn_path}")